In [150]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
import xlrd

### Loading the data
Dataset cargado en el DataFrame df_raw: todas las columnas aparecen en un único string.  
Las instrucciones para separar los datos en columnas están recogidas en el fichero disreg_recurciencia.xls. Las columnas relevantes para asignar nombres a las variables y establecer los caracteres correspondientes se cargan en el Dataframe df_aux. 

In [337]:
df_raw=pd.read_csv('/home/bea/Escritorio/Encuesta_RRHH_Ciencia_Tecnología/RRHH09.txt',header=0,names=['string'])
df_raw.head()

,string
0,01000416196516EU271EU27NNNNNNNN21 0 2 07 8 122222211612NN 0NN 013031111992 9 311EU27198810 . .1418147200016EU271226 1 1 33311111432111110 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
1,01000616197213EU271EU27NNNNNNNN21 0 0 08 7 151N33111612NN 0NN 01104110200011 211EU27199511 0 02 . NN .NNNNNN NNN NNN . . . . . 111111 1 1 NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
2,01000716196816EU271EU27NNNNNNNN21 1 1 02 2 155N11111612NN 0NN 015041102001 6 9 2EU27199161 1 11405185200517EU27223124 1 22122111212111110 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
3,01000816196916EU271EU27NNNNNNNN26 0 2 03 3 127912111612NN 0NN 012070112002 2 8 5EU27199361 0 11376085199616EU2732314311 22112111112111110 NNN . .0 . . . EU2720062009 48EU2720052005 7EU2720002005 65NMOV . . .NMOV . . .NMOV . . .NMOV . . . 1 ...
4,01000916197516EU271EU27NNNNNNNN21 0 0 04 3 157N22211612NN 0NN 013031002002 6 4 0EU27199810 . .1408021200216EU271226 211 33222222211222220 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...


In [338]:
df_aux=pd.read_excel('/home/bea/Escritorio/Encuesta_RRHH_Ciencia_Tecnología/disreg_recurciencia.xls',
                    sheet_name='RRHH09',skiprows=2,usecols=['Nemotécnico','Fin'],skipfooter=6)
df_aux.rename(columns={'Nemotécnico':'Col_name'},inplace=True) # Columna de nombres de las variables de df_raw
df_aux['Fin']=df_aux['Fin'].astype('int') # Columna de último caracter correspondiente a cada variable (numerados desde 1)

In [339]:
df_aux.head()

,Col_name,Fin
0,MUIDENT,6
1,CCAARESI,8
2,ANONAC,12
3,CCAANAC,14
4,CONTNACIM,18


In [340]:
df_aux.tail()

,Col_name,Fin
194,COMPA,356
195,TUTOR,357
196,COOPERA,358
197,INVFUT,359
198,FACTOR,370


### Retrieving the variables 
Cada fila de df_aux corresponde a una variable (o columna) del dataset: incluye nombre de la variable y posicion del último caracter. 
Para separar el string único df_raw['string'] en columnas y asignarle a cada una su denominacion original: 

In [341]:
inicio=0
for i in range(0,199):
    df_raw[df_aux['Col_name'].iloc[i]]=df_raw['string'].str[inicio:df_aux['Fin'].iloc[i]]
    inicio=df_aux['Fin'].iloc[i]
del df_raw['string'] 
df_raw.head()

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR
0,010004,16,1965,16,EU27,1,EU27,NNNN,NNNN,2,1,0,2,0,7,8,1,2,2,2,2,2,2,1,16,12,NN,0,NN,0,1,303,1,1,1,1992,9,3,11,EU27,1988,1,0,.,.,1,41,8,1,47,2000,16,EU27,1,226,,1,,1,,3,3,3,1,1,1,1,1,4,3,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,13.04720815
1,010006,16,1972,13,EU27,1,EU27,NNNN,NNNN,2,1,0,0,0,8,7,1,5,1,N,3,3,1,1,16,12,NN,0,NN,0,1,104,1,1,0,2000,11,2,11,EU27,1995,1,1,0,0,2,.,,,NN,.,NN,NNNN,,NNN,,,,,,,,,,,,,,,,,,,,,,,,NNN,.,.,,,,,,,,,,.,.,.,,1,1,1,1,1,1,,1,,1,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,1,,,,,,,,1,1,,,,,,,,,36,0,0,0,0,0,0,0,0,0,1,0,13.04720815
2,010007,16,1968,16,EU27,1,EU27,NNNN,NNNN,2,1,1,1,0,2,2,1,5,5,N,1,1,1,1,16,12,NN,0,NN,0,1,504,1,1,0,2001,6,9,2,EU27,1991,6,1,1,1,1,40,5,1,85,2005,17,EU27,2,231,2,4,,1,,2,2,1,2,2,1,1,1,2,1,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,,,,,40,4,2,0,0,0,0,0,0,0,1,1,13.04720815
3,010008,16,1969,16,EU27,1,EU27,NNNN,NNNN,2,6,0,2,0,3,3,1,2,7,9,1,2,1,1,16,12,NN,0,NN,0,1,207,0,1,1,2002,2,8,5,EU27,1993,6,1,0,1,1,37,6,0,85,1996,16,EU27,3,231,4,3,1,1,,2,2,1,1,2,1,1,1,1,1,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,EU27,2006,2009,48,EU27,2005,2005,7,EU27,2000,2005,65,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,1,,,,,1,,,,,,2,5,,,1,,,,EUCA,1,,,,,,,,,,,,,,,,,,1,,120,3,10,0,0,0,0,0,0,0,1,1,13.04720815
4,010009,16,1975,16,EU27,1,EU27,NNNN,NNNN,2,1,0,0,0,4,3,1,5,7,N,2,2,2,1,16,12,NN,0,NN,0,1,303,1,0,0,2002,6,4,0,EU27,1998,1,0,.,.,1,40,8,0,21,2002,16,EU27,1,226,,2,1,1,,3,3,2,2,2,2,2,2,2,1,1,2,2,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,13.04720815


In [342]:
df_raw.tail()

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR
4117,520044,18,1949,01,EU27,1,EU27,NNNN,NNNN,1,4,0,0,1,3,3,2,5,0,N,1,1,1,1,01,12,NN,0,NN,0,1,602,0,1,0,2000,10,5,2,EU27,1994,6,1,3,2,3,.,,,NN,.,NN,NNNN,,NNN,,,,,,,,,,,,,,,,,,,,,,,,NNN,.,.,,,,,,,,,,.,.,.,,0,,,,,,,,,,,,,,,,1,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,72,0,1,0,0,0,0,0,0,1,1,1,2.557220801
4118,520046,18,1973,01,EU27,1,EU27,NNNN,NNNN,1,1,0,2,0,7,6,1,1,2,2,3,3,3,1,18,9,01,3,NN,0,1,601,1,1,1,2003,9,7,1,EU27,1996,8,1,3,3,1,30,3,0,85,2003,18,EU27,2,233,,5,1,1,,3,7,1,1,1,1,1,2,4,1,1,2,1,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,2,8,,,1,,,,EUCA,0,,1,,1,,,,,,1,,,,,,,,1,,50,10,3,1,1,0,0,0,0,0,1,1,2.557220801
4119,520047,13,1958,18,EU27,1,EU27,NNNN,NNNN,1,6,0,0,0,6,6,2,2,4,4,2,2,2,1,13,12,NN,0,NN,0,1,601,1,1,0,1994,9,2,9,EU27,1990,8,1,1,2,1,40,2,0,85,2009,13,EU27,3,231,3,4,2,1,,3,3,2,2,2,2,2,2,2,2,2,1,2,2,1,2,1,4,262,10,1,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,240,8,3,0,0,0,0,0,0,0,1,1,2.557220801
4120,520048,18,1967,10,EU27,1,EU27,NNNN,NNNN,1,1,0,1,1,6,2,2,5,2,N,3,1,1,1,18,12,NN,0,NN,0,1,503,1,0,0,2002,2,6,0,EU27,1994,8,1,0,0,1,70,8,0,85,1995,01,EU27,3,231,2,3,1,1,,2,2,2,1,1,2,3,3,2,1,1,1,1,1,1,2,0,,NNN,.,.,1,0,0,1,1,1,0,1,0,25,25,50,3,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,2,4,,,1,,,,EUCA,1,,,,,,,,,,,1,1,1,1,1,1,,,,192,50,60,1,0,1,0,1,0,1,1,1,2.557220801
4121,520050,18,1967,13,EU27,1,EU27,NNNN,NNNN,1,6,0,0,0,6,2,2,5,9,N,1,1,1,1,18,12,NN,0,NN,0,1,602,1,0,0,2004,1,8,0,EU27,1996,8,0,.,.,1,35,5,0,85,1998,18,EU27,2,233,,5,2,1,,3,3,3,2,2,2,1,1,3,3,2,2,2,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,1,,1,1,,1,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,2.557220801


In [343]:
df_raw.shape

(4122, 199)

In [582]:
df_raw.to_csv('/home/bea/Escritorio/Encuesta_RRHH_Ciencia_Tecnología/raw_doct.csv')

### Variable description, types and missing values

In [158]:
df_doct=df_raw

#### 'MUIDENT'
Índice único del entrevistado.

In [159]:
df_doct['MUIDENT']=df_doct['MUIDENT'].astype('int') # No missing values

#### 'CCAARESI'
CCAA de residencia el 31 de diciembre de 2009: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de),
'04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [160]:
df_doct['CCAARESI']=df_doct['CCAARESI'].astype('int') # No missing values
df_doct['CCAARESI'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

#### 'ANONAC'
Año de nacimiento.

In [161]:
df_doct['ANONAC']=df_doct['ANONAC'].astype('int') # No missing values
df_doct['ANONAC'].sort_values().unique()

array([1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980])

#### 'CCAANAC'
CCAA de nacimiento (Códigos: vease variable 'CCAARESI').

In [162]:
df_doct['CCAANAC'].unique() # blank as 'NN' when no answer expected 

array(['16', '13', 'NN', '12', '15', '08', '07', '14', '10', '09', '01',
       '11', '04', '02', '03', '06', '18', '05', '17'], dtype=object)

In [163]:
df_doct['CCAANAC'].replace({'NN':None},inplace=True) # None when not born in Spain
df_doct['CCAANAC'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', None], dtype=object)

#### 'CONTNACIM'
Continente de nacimiento: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [164]:
df_doct['CONTNACIM'].unique() # No missing values

array(['EU27', 'RAME', 'REUR', 'EUCA', 'ASIA', 'AFRI', 'OCEA'],
      dtype=object)

#### 'RELA'
Relación que le une con España: '1'=Español por nacimiento, '2'=Español por nacionalización, '3'=Residente permanente, '4'=Residente temporal, '5'=Refugiado.

In [165]:
df_doct['RELA']=df_doct['RELA'].astype('int') # No missing values
df_doct['RELA'].unique()

array([1, 2, 3, 4])

#### 'CONTNAC1'
Continente del pais de su primera nacionalidad (Códigos: vease variable 'CONTNACIM').

In [166]:
df_doct['CONTNAC1'].unique() # No missing values

array(['EU27', 'RAME', 'REUR', 'EUCA', 'AFRI', 'ASIA'], dtype=object)

#### 'CONTNAC2'
Continente del pais de su segunda nacionalidad (Códigos: vease variable 'CONTNACIM').

In [167]:
df_doct['CONTNAC2'].unique() # Blank as 'NNNN' when no answer expected

array(['NNNN', 'EU27', 'EUCA', 'RAME', 'AFRI', 'REUR'], dtype=object)

In [168]:
df_doct['CONTNAC2'].replace({'NNNN':None},inplace=True) # None when no second nationality
df_doct['CONTNAC2'].unique()

array([None, 'EU27', 'EUCA', 'RAME', 'AFRI', 'REUR'], dtype=object)

#### 'CONTNAC3'
Continente del pais de su tercera nacionalidad (Códigos: vease variable 'CONTNACIM').

In [169]:
df_doct['CONTNAC3'].unique() # Blank as 'NNNN' when no answer is expected

array(['NNNN', 'RAME'], dtype=object)

In [170]:
df_doct['CONTNAC3'].replace({'NNNN':None},inplace=True) # None when no third nationality
df_doct['CONTNAC3'].unique()

array([None, 'RAME'], dtype=object)

#### 'SEXO'
Sexo: '1'=Hombre, '2'='Mujer'

In [171]:
df_doct['SEXO']=df_doct['SEXO'].astype('int') # No missing values 
df_doct['SEXO'].unique() 

array([2, 1])

#### 'ESTADOCIVIL'
Estado civil: '1'=Casado, '2'=Pareja de hecho, '3'=Separado, '4'=Divorciado, '5'=Viudo, '6'=Soltero.

In [172]:
df_doct['ESTADOCIVIL']=df_doct['ESTADOCIVIL'].astype('int') # No missing values
df_doct['ESTADOCIVIL'].sort_values().unique()

array([1, 2, 3, 4, 5, 6])

#### 'DEPEN5'
Número de personas dependientes a su cargo de 5 años o menos.  

In [173]:
df_doct['DEPEN5']=df_doct['DEPEN5'].astype('int') # No missing values
df_doct['DEPEN5'].sort_values().unique()

array([0, 1, 2, 3, 5])

#### 'DEPEN18'
Número de personas dependientes a su cargo de 6 a 18 años.

In [174]:
df_doct['DEPEN18']=df_doct['DEPEN18'].astype('int') # No missing values
df_doct['DEPEN18'].sort_values().unique()

array([0, 1, 2, 3, 4, 5, 6])

#### 'DEPENMAS'
Número de personas dependientes a su cargo de más de 18 años.

In [175]:
df_doct['DEPENMAS']=df_doct['DEPENMAS'].astype('int') # No missing values
df_doct['DEPENMAS'].sort_values().unique()

array([0, 1, 2, 3, 4, 6])

#### 'NIVESTPA'
Nivel educativo del padre: '1'=Sin estudios, '2'=Enseñanza Primaria Incompleta, '3'=ESO,EGB,Primaria completa o equivalente, '4'=FP I, '5'=FP II, '6'=Bachiller Superior,BUP,COU, '7'=Diplomado,Arquitecto o Ingeniero Técnico o equiv, '8'=Licenciado,Arquitecto o Ingeniero Superior o equiv, '9'=Doctorado,postgrado o especialización para licenciados, '10'=NS/NC.

In [176]:
df_doct['NIVESTPA']=df_doct['NIVESTPA'].astype('int') # No missing values (NS/NC as value '10')
df_doct['NIVESTPA'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [177]:
df_doct['NIVESTPA'].value_counts()

3     866
2     848
8     657
7     506
6     340
9     316
1     267
5     166
4     123
10     33
Name: NIVESTPA, dtype: int64

In [178]:
# Sustituyo NS/NC por None (evita que sea interpretada como el nivel de estudios más alto recogido)
df_doct['NIVESTPA'].replace({10:None},inplace=True)
df_doct['NIVESTPA']=df_doct['NIVESTPA'].astype('float')
df_doct['NIVESTPA'].sort_values().unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., nan])

#### 'NIVESTMA'
Nivel educativo de la madre (Códigos: véase la variable 'NIVESTPA').

In [179]:
df_doct['NIVESTMA']=df_doct['NIVESTMA'].astype('int') # No missing values
df_doct['NIVESTMA'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [180]:
df_doct['NIVESTMA'].value_counts()

3     1247
2     1137
7      445
6      403
1      363
8      298
9       81
4       71
5       54
10      23
Name: NIVESTMA, dtype: int64

In [181]:
# Sustituyo NS/NC por None (evita que sea interpretada como el nivel de estudios más alto alcanzable)
df_doct['NIVESTMA'].replace({10:None},inplace=True)
df_doct['NIVESTMA']=df_doct['NIVESTMA'].astype('float')
df_doct['NIVESTMA'].sort_values().unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., nan])

#### 'NIVPROFPA'
Sector al que pertenece la principal organización en la que ha trabajado el padre del encuestado: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL, 5=No ha trabajado nunca, 6=otros.

In [182]:
df_doct['NIVPROFPA']=df_doct['NIVPROFPA'].astype('int') # No missing values
df_doct['NIVPROFPA'].sort_values().unique()

array([1, 2, 3, 4, 6])

#### 'NIVPROFMA'
Sector al que pertenece la principal organización en la que ha trabajado la madre del encuestado (Códigos: véase la variable 'NIVPROFPA').

In [183]:
df_doct['NIVPROFMA']=df_doct['NIVPROFMA'].astype('int') # No missing values
df_doct['NIVPROFMA'].sort_values().unique()

array([1, 2, 3, 4, 5, 6])

#### 'ACTIPRIPA1'
Sector de actividad laboral principal del padre: Código de actividad laboral válido a 1 dígito o blanco

In [184]:
df_doct['ACTIPRIPA1']=df_doct['ACTIPRIPA1'].astype('int') # No missing values
df_doct['ACTIPRIPA1'].sort_values().unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

#### 'ACTIPRIMA1'
Sector de actividad laboral principal de la madre: Código de actividad laboral válido a 1 dígito o blanco

In [185]:
df_doct['ACTIPRIMA1'].unique() # Blank as 'N' 

array(['2', 'N', '9', '5', '1', '4', '7', '6', '3', '8', '0'],
      dtype=object)

In [186]:
# Blank when no answer is expected (no paid activity). No missing values
df_doct[['NIVPROFMA','ACTIPRIMA1']].groupby('NIVPROFMA').agg(['unique'])

,ACTIPRIMA1
,unique
NIVPROFMA,
1,"[5, 4, 2, 9, 7, 6, 3, 8, 1, 0]"
2,"[2, 9, 1, 4, 5, 3]"
3,"[2, 3, 1]"
4,"[2, 9, 4, 5]"
5,[N]
6,[2]


In [187]:
df_doct['ACTIPRIMA1'].replace({'N':None},inplace=True) # None when no paid activity (no missing values)
df_doct['ACTIPRIMA1']=df_doct['ACTIPRIMA1'].astype('float')  
df_doct['ACTIPRIMA1'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., nan])

#### 'EDUPRIM'
Tipo de centro en que cursó la educación primaria: 1=Centro Público, 2=Centro Concertado, 3=Centro Privado.

In [188]:
df_doct['EDUPRIM']=df_doct['EDUPRIM'].astype('int') # No missing values
df_doct['EDUPRIM'].sort_values().unique()

array([1, 2, 3])

#### 'EDUSECUN'
Tipo de centro en que cursó la educacion secundaria o la FP II (Códigos: vease la variable 'EDUPRIM').

In [189]:
df_doct['EDUSECUN']=df_doct['EDUSECUN'].astype('int') # No missing values
df_doct['EDUSECUN'].sort_values().unique()

array([1, 2, 3])

#### 'BACHILLER'
Tipo de centro en que cursó el bachillerato (Códigos: vease la variable 'EDUPRIM').

In [190]:
df_doct['BACHILLER']=df_doct['BACHILLER'].astype('int') # No missing values
df_doct['BACHILLER'].sort_values().unique()

array([1, 2, 3])

#### 'RES'
¿Residió en España en algún periodo de 2009? 0=No, 1=Sí. 

In [191]:
df_doct['RES']=df_doct['RES'].astype('int') # No missing values
df_doct['RES'].unique()

array([1, 0])

#### 'CARES1'
Primera CCAA de residencia en 2009 (Códigos: véase la variable 'CCAARESI').

In [192]:
df_doct['CARES1'].unique() # Blank as 'NN' when no answer is expected 

array(['16', 'NN', '13', '09', '08', '01', '07', '10', '11', '04', '05',
       '06', '12', '03', '14', '02', '17', '15', '18'], dtype=object)

In [193]:
df_doct[['RES','CARES1']].groupby('RES').agg(['unique']) # 'NN' when not lived in Spain during 2009 

,CARES1
,unique
RES,
0,[NN]
1,"[16, 13, 09, 08, 01, 07, 10, 11, 04, 05, 06, 12, 03, 14, 02, 17, 15, 18]"


In [194]:
df_doct['CARES1'].replace({'NN':None},inplace=True) 
df_doct['CARES1'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', None], dtype=object)

#### 'MESRES1'
Duración en meses del periodo de 2009 en que residió en la primera CCAA identificada.

In [195]:
df_doct['MESRES1'].unique() # Blank as ' .' 

array(['12', ' .', ' 8', '10', ' 6', ' 9', '11', ' 7', ' 5', ' 3', ' 4',
       ' 1', ' 2'], dtype=object)

In [196]:
df_doct[['RES','MESRES1']].groupby('RES').agg(['unique']) # ' .' when not lived in Spain during 2009 (no missing values)

,MESRES1
,unique
RES,
0,[ .]
1,"[12, 8, 10, 6, 9, 11, 7, 5, 3, 4, 1, 2]"


In [197]:
df_doct['MESRES1'].replace({' .':None},inplace=True) # None when not lived in Spain during 2009
df_doct['MESRES1']=df_doct['MESRES1'].astype('float')
df_doct['MESRES1'].sort_values().unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., nan])

In [198]:
df_doct[['RES','MESRES1']].groupby('RES').agg(['unique'])

,MESRES1
,unique
RES,
0,[nan]
1,"[12.0, 8.0, 10.0, 6.0, 9.0, 11.0, 7.0, 5.0, 3.0, 4.0, 1.0, 2.0]"


#### 'CARES2'
Segunda CCAA de residencia en 2009 (Códigos: véase la variable 'CCAARESI').

In [199]:
df_doct['CARES2'].unique() # Blank as 'NN' 

array(['NN', '08', '10', '13', '01', '09', '11', '04', '03', '02', '05',
       '16', '12', '07', '17', '06', '15', '18'], dtype=object)

In [200]:
df_doct['CARES2'].replace({'NN':'None'},inplace=True) # 'None' when not lived in two different CCAA during 2009
# 'None' as a string (above) to allow for checking the next variable values associated to this question being not answered
df_doct['CARES2'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '15', '16', '17', '18', 'None'], dtype=object)

#### 'MESRES2'
Duración en meses del periodo de 2009 en que residió en la segunda CCAA identificada.

In [201]:
df_doct['MESRES2'].unique() # Blank as ' .' 

array([' 0', ' .', ' 6', ' 4', ' 1', ' 2', ' 3', ' 7', ' 9', ' 5', '10',
       '11', ' 8'], dtype=object)

In [202]:
df_doct[['CARES2','MESRES2']].groupby('CARES2').agg(['unique']) # Blank as ' .' or ' 0' when no answer is expected

,MESRES2
,unique
CARES2,
01,"[ 4, 1, 5, 6, 2, 3, 7, 8, 10]"
02,"[ 4, 6, 3, 10, 1, 8, 11]"
03,"[ 4, 1, 2, 3, 6]"
04,"[ 1, 6, 9, 3]"
05,"[ 2, 3, 9, 1]"
06,"[ 2, 10, 1, 3, 5]"
07,"[ 1, 2, 9, 5, 6, 4]"
08,"[ 6, 7, 3, 2, 1, 4]"


In [203]:
df_doct['MESRES2'].replace({' .':0},inplace=True) # 0 when not lived in two different CCAA during 2009
df_doct['MESRES2']=df_doct['MESRES2'].astype('int')
df_doct['MESRES2'].sort_values().unique() # CCAA not sorted according to duration of residence (some values of 'MESRES2' greater than 6)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [204]:
df_doct[['CARES2','MESRES2']].groupby('CARES2').agg(['unique'])

,MESRES2
,unique
CARES2,
01,"[4, 1, 5, 6, 2, 3, 7, 8, 10]"
02,"[4, 6, 3, 10, 1, 8, 11]"
03,"[4, 1, 2, 3, 6]"
04,"[1, 6, 9, 3]"
05,"[2, 3, 9, 1]"
06,"[2, 10, 1, 3, 5]"
07,"[1, 2, 9, 5, 6, 4]"
08,"[6, 7, 3, 2, 1, 4]"


#### 'CARES3'
Tercera CCAA de residencia en 2009 (Códigos: véase la variable 'CCAARESI').

In [205]:
df_doct['CARES3'].unique() # Blank as 'NN' 

array(['NN', '01', '12', '09', '03', '13', '07'], dtype=object)

In [206]:
df_doct['CARES3'].replace({'NN':'None'},inplace=True) # 'None' when not lived in three different CCAA during 2009
# 'None' as a string (above) to allow for checking the next variable values associated to this question being not answered
df_doct['CARES3'].sort_values().unique() 

array(['01', '03', '07', '09', '12', '13', 'None'], dtype=object)

#### 'MESRES3'
Duración en meses del periodo de 2009 en que residió en la tercera CCAA identificada.

In [207]:
df_doct['MESRES3'].unique() # Blank as ' .'

array([' 0', ' .', ' 2', ' 3', ' 1', '10'], dtype=object)

In [208]:
df_doct[['CARES3','MESRES3']].groupby('CARES3').agg(['unique']) # Blank as ' .' when no answer is expected

,MESRES3
,unique
CARES3,
01,"[ 2, 1]"
03,"[ 3, 1]"
07,[ 1]
09,"[ 2, 1]"
12,"[ 3, 2]"
13,"[10, 2, 1]"
None,"[ 0, .]"


In [209]:
df_doct['MESRES3'].replace({' .':0},inplace=True) # 0 when not lived in three different CCAA during 2009
df_doct['MESRES3']=df_doct['MESRES3'].astype('int')
df_doct['MESRES3'].sort_values().unique() 

array([ 0,  1,  2,  3, 10])

In [210]:
df_doct[['CARES3','MESRES3']].groupby('CARES3').agg(['unique'])

,MESRES3
,unique
CARES3,
01,"[2, 1]"
03,"[3, 1]"
07,[1]
09,"[2, 1]"
12,"[3, 2]"
13,"[10, 2, 1]"
None,[0]


#### 'DOCTOR'
Título de doctor anterior al 31 de diciembre de 2009: 0=No, 1=Sí

In [211]:
df_doct['DOCTOR']=df_doct['DOCTOR'].astype('int') # No missing values
df_doct['DOCTOR'].unique()

array([1])

In [212]:
del df_doct['DOCTOR'] # As all observations have the same value for 'DOCTOR'

#### 'CODIDOC'
Código de estudios de doctorado:  

CIENCIAS NATURALES: 101=Matemáticas, 102=Informática y tecnologías de la información, 103=Ciencias físicas, 104=Ciencias químicas, 105=Ciencias de la tierra y medio ambiente, 106=Biología (excluyendo agricultura y ciencias médicas), 107=Otras ciencias naturales  

INGENIERÍA Y TECNOLOGÍA: 201=Ingeniería civil, 202=Ingeniería eléctrica, electrónica y de telecomunicaciones, 203=Ingeniería mecánica, 204=Ingeniería química, 205=Ingeniería de materiales, 206=Ingeniería médica, 207=Ingeniería medioambiental, 208=Biotecnología medioambiental, 209=Biotecnología industrial, 210=Nanotecnología, 211=Otras ingenierías y tecnologías (comida, bebida y otras)  

CIENCIAS MÉDICAS: 301=Medicina básica, 302=Medicina clínica, 303=Ciencias de la salud, 304=Biotecnología médica, 305=Otras ciencias médicas (forenses y otras ciencias médicas)  

CIENCIAS DE LA AGRICULTURA: 401=Agricultura, ciencias forestales y piscifactorías, 402=Ciencias de los animales y de la leche, 403=Veterinaria, 404=Biotecnología agrícola, 405=Otras ciencias de la agricultura  

CIENCIAS SOCIALES: 501=Psicología, 502=Economía y empresas, 503=Ciencias de la educación, 504=Sociología, 505=Derecho, 506=Ciencias políticas, 507=Geografía económica y social, 508=Periodismo y comunicaciones, 509=Otras ciencias sociales  

HUMANIDADES: 601=Historia y arqueología, 602=Lenguaje y literatura, 603=Filosofía, ética y religión, 604=Arte (historia del arte, bellas artes y música), 605=Otras humanidades  

In [213]:
df_doct['CODIDOC']=df_doct['CODIDOC'].astype('int') # No missing values
df_doct['CODIDOC'].sort_values().unique()

array([101, 102, 103, 104, 105, 106, 107, 201, 202, 203, 204, 205, 206,
       207, 208, 209, 210, 211, 301, 302, 303, 304, 305, 401, 402, 403,
       404, 405, 501, 502, 503, 504, 505, 506, 507, 508, 509, 601, 602,
       603, 604, 605])

#### 'INFUN'
Carecter de investigación básica o fundamental de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [214]:
df_doct['INFUN']=df_doct['INFUN'].astype('int') # No missing values
df_doct['INFUN'].unique()

array([1, 0])

#### 'INAPL'
Carecter de investigación aplicada de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [215]:
df_doct['INAPL']=df_doct['INAPL'].astype('int') # No missing values
df_doct['INAPL'].unique()

array([1, 0])

#### 'DESTEC'
Carecter de desarrollo experimental de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [216]:
df_doct['DESTEC']=df_doct['DESTEC'].astype('int') # No missing values
df_doct['DESTEC'].unique()

array([1, 0])

#### 'ANODOC'
Año en que obtuvo el título de doctor.

In [217]:
df_doct['ANODOC']=df_doct['ANODOC'].astype('int') # No missing values
df_doct['ANODOC'].sort_values().unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009])

#### 'MESDOC'
Mes en que obtuvo el título de doctor.

In [218]:
df_doct['MESDOC']=df_doct['MESDOC'].astype('int') # No missing values
df_doct['MESDOC'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

#### 'DURDOCANO'
Tiempo transcurrido desde que inició los cursos de doctorado hasta que obtuvo el título de doctor: años completos. 

In [219]:
df_doct['DURDOCANO']=df_doct['DURDOCANO'].astype('int') # No missing values
df_doct['DURDOCANO'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 30, 35])

#### 'DURDOCMES'
Tiempo transcurrido desde que inició los cursos de doctorado hasta que obtuvo el título de doctor: meses adicionales a 'DURDOCANO'.

In [220]:
df_doct['DURDOCMES']=df_doct['DURDOCMES'].astype('int') # No missing values
df_doct['DURDOCMES'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [221]:
df_doct[df_doct['DURDOCMES']==12] # OBS TO BE RECONSIDERED:

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR
307,70096,4,1964,04,EU27,1,EU27,None,None,1,1,0,2,0,3.0,3.0,1,1,9,9.0,1,1,1,1,04,12.0,None,0,None,0,101,1,1,0,1994,7,7,12,EU27,1987,5,1,1,10,1,40,5,0,85,1989,04,EU27,3,231,2,3,1,1,,2,2,3,3,3,1,1,1,3,2,1,1,2,3,1,3,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,,,,,200,5,0,0,0,0,0,0,0,0,0,1,11.51173913
1297,190035,8,1965,12,EU27,1,EU27,None,None,2,6,0,0,0,7.0,7.0,2,2,2,2.0,1,1,1,0,None,NaN,None,0,None,0,605,1,0,0,1995,6,1,12,EU27,1994,8,0,.,.,1,40,5,0,01,2007,08,EU27,1,100,,2,1,1,,3,3,3,2,3,2,1,1,2,2,2,2,1,2,2,2,1,1,100,1,3,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,15.26447936
1485,250034,9,1963,09,EU27,1,EU27,None,None,1,1,0,2,0,2.0,8.0,1,2,4,2.0,2,1,1,1,09,12.0,None,0,None,0,302,1,1,0,1998,9,4,12,EU27,1988,2,0,.,.,1,50,3,0,86,1996,09,EU27,2,221,,1,1,1,,3,2,3,3,3,2,3,3,4,4,1,2,1,3,2,3,1,1,224,18,0,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,1,,,,,,,48,2,0,0,0,0,0,0,0,0,0,0,25.66619877
2405,290083,1,1968,11,EU27,1,EU27,None,None,1,1,0,1,1,1.0,1.0,2,5,6,NaN,1,1,1,1,01,12.0,None,0,None,0,502,0,1,0,2006,4,10,12,EU27,1992,5,1,0,0,1,45,5,0,85,1995,01,EU27,2,231,3,4,1,1,,2,1,1,2,3,2,1,3,3,1,2,3,2,3,1,1,0,,NNN,.,.,1,0,0,1,0,1,0,0,1,25,65,10,3,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,1,,,,,1,,30,2,0,0,0,0,0,0,0,0,0,1,25.66800106


In [222]:
# New variable (not in the original dataset):
df_doct['Tmpo_doc']=df_doct['DURDOCANO']+df_doct['DURDOCMES']/12
df_doct['Tmpo_doc'].sort_values().unique()

array([ 0.        ,  0.5       ,  1.        ,  1.08333333,  1.5       ,
        1.75      ,  2.        ,  2.08333333,  2.16666667,  2.25      ,
        2.33333333,  2.41666667,  2.5       ,  2.58333333,  2.66666667,
        2.75      ,  2.83333333,  2.91666667,  3.        ,  3.08333333,
        3.16666667,  3.25      ,  3.33333333,  3.41666667,  3.5       ,
        3.58333333,  3.66666667,  3.75      ,  3.83333333,  3.91666667,
        4.        ,  4.08333333,  4.16666667,  4.25      ,  4.33333333,
        4.41666667,  4.5       ,  4.58333333,  4.66666667,  4.75      ,
        4.83333333,  4.91666667,  5.        ,  5.08333333,  5.16666667,
        5.25      ,  5.33333333,  5.41666667,  5.5       ,  5.58333333,
        5.66666667,  5.75      ,  5.83333333,  5.91666667,  6.        ,
        6.08333333,  6.16666667,  6.25      ,  6.33333333,  6.41666667,
        6.5       ,  6.58333333,  6.66666667,  6.75      ,  6.83333333,
        6.91666667,  7.        ,  7.08333333,  7.16666667,  7.25

In [223]:
df_doct[df_doct['Tmpo_doc']==0] # OBS TO BE RECONSIDERED:

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,Tmpo_doc
469,80195,9,1951,09,EU27,1,EU27,None,None,2,1,0,2,0,2.0,2.0,1,5,7,NaN,2,2,3,1,09,12.0,None,0,None,0,601,0,1,1,1991,1,0,0,EU27,1973,5,1,0,0,1,40,7,0,91,1992,09,EU27,2,262,,1,1,1,,3,3,1,2,2,1,1,1,1,1,2,2,2,3,2,3,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,20,0,0,0,0,0,0,0,0,0,1,1,25.66619877,0.0
2120,280690,13,1942,07,EU27,1,EU27,None,None,1,1,0,0,0,9.0,3.0,1,5,3,NaN,3,3,3,1,13,12.0,None,0,None,0,201,1,1,0,1992,10,0,0,EU27,1966,9,1,0,0,1,35,8,0,85,1997,13,EU27,3,231,2,3,1,1,,2,1,1,2,2,1,1,1,2,1,1,1,1,2,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,1,,1,,432,0,9,0,0,0,0,0,0,0,0,1,24.81791986,0.0


#### 'CONTEST'
Continente en el que realizó los estudios universitarios de segundo ciclo previos al doctorado (Códigos: véase la variable 'CONTNACIM').

In [224]:
df_doct['CONTEST'].sort_values().unique() # No missing values

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'RAME', 'REUR'], dtype=object)

#### 'ANOEST'
Año en el que finalizó los estudios universitarios de segundo ciclo.

In [225]:
df_doct['ANOEST']=df_doct['ANOEST'].astype('int') # No missing values
df_doct['ANOEST'].sort_values().unique()

array([1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004])

#### 'FINADOC'
Fuente principal de financiación durante los estudios de doctorado: 1=Beca de la Administración Pública (central, autonómica, local,etc.), 2=Beca de otro tipo de institución española, 3=Beca de una Administración Pública extranjera, 4=Beca de otro tipo de institución extranjera, 5=Trabajó como profersor y/o ayudante de investigación, 6=Otra ocupación, 7=Subvención reembolsada por el empleador, 8=Préstamos, ahorros personales y/o ayuda familiar, 9=Otras formas. 

In [226]:
df_doct['FINADOC']=df_doct['FINADOC'].astype('int') # No missing values
df_doct['FINADOC'].sort_values().unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

#### 'TRABDOC'
¿Trabajó en algo relacionado con su doctorado una vez finalizado y antes de enero de 2010?: 0=No, 1=Sí

In [227]:
df_doct['TRABDOC']=df_doct['TRABDOC'].astype('int') # No missing values
df_doct['TRABDOC'].sort_values().unique()

array([0, 1])

#### 'ANOSINTRA'
(Solo para respuesta positiva en variable 'TRABDOC')  
Tiempo transcurrido desde la finalización de los estudios de doctorado hasta que empezó a trabajar en algo relacionado con estos: años completos. 

In [228]:
df_doct['ANOSINTRA'].sort_values().unique() # Blank as ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '12', '14', '15', '16', '17', '19'], dtype=object)

In [229]:
df_doct[['ANOSINTRA','TRABDOC']].groupby('TRABDOC').agg(['unique']) # No missing values

,ANOSINTRA
,unique
TRABDOC,
0,[ .]
1,"[ 0, 1, 2, 4, 3, 14, 5, 16, 7, 15, 6, 9, 10, 8, 19, 12, 17]"


In [230]:
df_doct['ANOSINTRA'].replace({' .':None},inplace=True) # None when no answer is expected
df_doct['ANOSINTRA']=df_doct['ANOSINTRA'].astype('float')
df_doct['ANOSINTRA'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 12., 14.,
       15., 16., 17., 19., nan])

#### 'MESSINTRA'
(Solo para respuesta positiva en variable 'TRABDOC')   
Tiempo transcurrido desde la finalización de los estudios de doctorado hasta que empezó a trabajar en algo relacionado con estos: meses adicionales a 'ANOSINTRA'.

In [231]:
df_doct['MESSINTRA'].sort_values().unique() # Blank as ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '11', '12'], dtype=object)

In [232]:
df_doct[['MESSINTRA','TRABDOC']].groupby('TRABDOC').agg(['unique']) # No missing values

,MESSINTRA
,unique
TRABDOC,
0,[ .]
1,"[ 0, 1, 9, 8, 10, 3, 6, 5, 2, 7, 4, 11, 12]"


In [233]:
df_doct['MESSINTRA'].replace({' .':None},inplace=True) # None when no answer is expected
df_doct['MESSINTRA']=df_doct['MESSINTRA'].astype('float')
df_doct['MESSINTRA'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       nan])

In [234]:
# New variable (not in the original dataset):
df_doct['Tmpo_s/T']=df_doct['ANOSINTRA']+df_doct['MESSINTRA']/12
df_doct['Tmpo_s/T'].sort_values().unique() # NULL VALUES (TO BE RECONSIDERED):

array([ 0.        ,  0.08333333,  0.16666667,  0.25      ,  0.33333333,
        0.41666667,  0.5       ,  0.58333333,  0.66666667,  0.75      ,
        0.83333333,  0.91666667,  1.        ,  1.08333333,  1.16666667,
        1.25      ,  1.33333333,  1.41666667,  1.5       ,  1.58333333,
        1.66666667,  1.75      ,  1.83333333,  1.91666667,  2.        ,
        2.08333333,  2.16666667,  2.25      ,  2.33333333,  2.41666667,
        2.5       ,  2.58333333,  2.66666667,  2.75      ,  2.83333333,
        2.91666667,  3.        ,  3.16666667,  3.25      ,  3.33333333,
        3.41666667,  3.5       ,  3.58333333,  3.66666667,  3.75      ,
        3.83333333,  3.91666667,  4.        ,  4.08333333,  4.16666667,
        4.25      ,  4.33333333,  4.41666667,  4.58333333,  4.83333333,
        5.        ,  5.08333333,  5.16666667,  5.25      ,  5.33333333,
        5.5       ,  5.66666667,  6.        ,  6.16666667,  6.5       ,
        6.75      ,  7.        ,  7.16666667,  7.25      ,  7.75

#### 'SITLAB'
Situación laboral el 31 de diciembre de 2009: 1=Empleado, 2=Desempleado, 3=Inactivo.

In [235]:
df_doct['SITLAB']=df_doct['SITLAB'].astype('int') # No missing values
df_doct['SITLAB'].unique()

array([1, 2, 3])

#### 'HTRAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Número medio de horas semanales trabajadas en diciembre de 2009. 

In [236]:
df_doct['HTRAB'].sort_values().unique() # Blank as ' .'

array([' .', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '20', '21', '24', '25', '26', '27',
       '28', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '62',
       '63', '64', '65', '68', '70', '71', '72', '75', '77', '80', '82',
       '84', '90', '98', '99'], dtype=object)

In [237]:
df_doct[['SITLAB','HTRAB']].groupby('SITLAB').agg(['unique']) # No missing values

,HTRAB
,unique
SITLAB,
1,"[41, 40, 37, 50, 45, 30, 35, 55, 28, 52, 90, 42, 60, 38, 5, 36, 70, 12, 44, 15, 49, 25, 48, 34, 46, 56, 10, 32, 47, 18, 33, 51, 24, 84, 17, 39, 58, 20, 82, 26, 43, 65, 16, 11, 8, 75, 80, 77, 72, 6, 99, 68, 63, 53, 27, 64, 62, 7, 31, 57, 9, 21, 13, 98, 59, 14, 71, 54, 4]"
2,[ .]
3,[ .]


In [238]:
df_doct['HTRAB'].replace({' .':0},inplace=True) # 0 for 'SITLAB' different from 1
df_doct['HTRAB']=df_doct['HTRAB'].astype('int')
df_doct['HTRAB'].sort_values().unique()

array([ 0,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20,
       21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
       58, 59, 60, 62, 63, 64, 65, 68, 70, 71, 72, 75, 77, 80, 82, 84, 90,
       98, 99])

In [379]:
def re_sitlab(col,blank,value=None,tipus='float'):
    print (df_doct[col].sort_values().unique())
    print (df_doct[['SITLAB',col]].groupby('SITLAB').agg(['unique']))
    df_doct[col].replace({blank:value},inplace=True) 
    df_doct[col]=df_doct[col].astype(tipus)
    print (df_doct[col].sort_values().unique())
    return 

#### 'INGRESOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Tramo en que se encontraban sus ingresos brutos anuales en 2009: 1=Menos de 10.000 euros, 2=De 10.000 a 20.000 euros, 3=De 20.001 a 30.000 euros, 4=De 30.001 a 35.000 euros, 5=De 35.001 a 40.000 euros, 6=De 40.001 a 45.000 euros, 7=De 45.001 a 50.000 euros, 8=Más de 50.000 euros. 

In [382]:
df_doct['INGRESOS'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4', '5', '6', '7', '8'], dtype=object)

In [383]:
re_sitlab('INGRESOS',blank=' ',value=0,tipus='int')

[' ' '1' '2' '3' '4' '5' '6' '7' '8']
                        INGRESOS
                          unique
SITLAB                          
1       [8, 5, 6, 3, 7, 2, 4, 1]
2                            [ ]
3                            [ ]
[0 1 2 3 4 5 6 7 8]


In [384]:
df_doct[['SITLAB','INGRESOS']].groupby('SITLAB').agg(['unique']) # No missing values

,INGRESOS
,unique
SITLAB,
1,"[8, 5, 6, 3, 7, 2, 4, 1]"
2,[0]
3,[0]


In [385]:
df_doct['INGRESOS'].dtypes

dtype('int64')

#### 'AUTON'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
¿El 31 de diciembre de 2009 su actividad laboral principal era trabajador por cuenta propia? 0=No, 1=Sí. 

In [386]:
df_doct['AUTON'].sort_values().unique() # Blank values as ' '

array([' ', '0', '1'], dtype=object)

In [387]:
re_sitlab('AUTON',blank=' ',value=0,tipus='int')

[' ' '0' '1']
         AUTON
        unique
SITLAB        
1       [1, 0]
2          [ ]
3          [ ]
[0 1]


#### 'CNAE2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Descripción de la actividad principal de la empresa en la que trabaja de acuerdo con la clasificación CNAE-2009. 

In [388]:
df_doct['CNAE2'].sort_values().unique() # Blank as 'NN'

array(['01', '02', '03', '10', '11', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '30', '32', '35', '36', '38', '41', '42',
       '43', '46', '47', '49', '51', '52', '53', '56', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72',
       '73', '74', '75', '77', '78', '79', '80', '81', '82', '84', '85',
       '86', '87', '88', '90', '91', '93', '94', '95', '96', '97', '99',
       'NN'], dtype=object)

In [389]:
re_sitlab('CNAE2',blank='NN')

['01' '02' '03' '10' '11' '19' '20' '21' '22' '23' '24' '25' '26' '27'
 '28' '30' '32' '35' '36' '38' '41' '42' '43' '46' '47' '49' '51' '52'
 '53' '56' '58' '59' '60' '61' '62' '63' '64' '65' '66' '68' '69' '70'
 '71' '72' '73' '74' '75' '77' '78' '79' '80' '81' '82' '84' '85' '86'
 '87' '88' '90' '91' '93' '94' '95' '96' '97' '99' 'NN']
                                                                                                                                                                                                                                                                           CNAE2
                                                                                                                                                                                                                                                                          unique
SITLAB                                                                                                           

#### 'ANOINIALP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Año en que empezó a trabajar en la organización.  

In [395]:
df_doct['ANOINIALP'].sort_values().unique() # Blank as '   .'

array(['   .', '1965', '1966', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009'], dtype=object)

In [396]:
re_sitlab('ANOINIALP',blank='   .')

['   .' '1965' '1966' '1968' '1969' '1970' '1971' '1972' '1973' '1974'
 '1975' '1976' '1977' '1978' '1979' '1980' '1981' '1982' '1983' '1984'
 '1985' '1986' '1987' '1988' '1989' '1990' '1991' '1992' '1993' '1994'
 '1995' '1996' '1997' '1998' '1999' '2000' '2001' '2002' '2003' '2004'
 '2005' '2006' '2007' '2008' '2009']
                                                                                                                                                                                                                                                                       ANOINIALP
                                                                                                                                                                                                                                                                          unique
SITLAB                                                                                                                               

#### 'CAEMPP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
CCAA en la que desarrolla su actividad laboral principal el 31 de diciembre de 2009 (Códigos: véase la variable 'CCAANAC'). 

In [401]:
df_doct['CAEMP'].sort_values().unique() # Blank as 'NN'

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', 'NN'], dtype=object)

In [402]:
re_sitlab('CAEMP',blank='NN')

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' 'NN']
                                                                               CAEMP
                                                                              unique
SITLAB                                                                              
1       [16, 17, 02, NN, 13, 09, 15, 08, 01, 10, 14, 07, 11, 04, 05, 06, 12, 03, 18]
2                                                                               [NN]
3                                                                               [NN]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 nan]


#### 'CONTEMP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Continente en la que desarrolla su actividad laboral principal el 31 de diciembre de 2009 (Códigos: véase la variable 'CONTINACIM'). 

In [411]:
df_doct['CONTEMP'].sort_values().unique() # Blank as 'NNNN'

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'NNNN', 'RAME', 'REUR'],
      dtype=object)

In [412]:
re_sitlab('CONTEMP',blank='NNNN',tipus='object')

['AFRI' 'ASIA' 'EU27' 'EUCA' 'NNNN' 'RAME' 'REUR']
                                     CONTEMP
                                      unique
SITLAB                                      
1       [EU27, EUCA, RAME, REUR, ASIA, AFRI]
2                                     [NNNN]
3                                     [NNNN]
['AFRI' 'ASIA' 'EU27' 'EUCA' 'RAME' 'REUR' None]


#### 'SECTOR'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Sector al que pertenece la organización en la que trabajaba el 31 de diciembre de 2009: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL.

In [421]:
df_doct['SECTOR'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4'], dtype=object)

In [422]:
re_sitlab('SECTOR',blank=' ')

[' ' '1' '2' '3' '4']
              SECTOR
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'CODALP'
(Solo para respuesta 'Empleado' en variable 'SITLAB')  
Actividad laboral que desarrollaba el 31 de diciembre de 2009 de acuerdo con la Clasificación internacional uniforme de ocupaciones (ISCO-08): 

100=Directores y gerentes

Profesionales científicos e intelectuales: 211=Físicos, químicos y profesionales afines, 212=Matemáticos, actuarios y estadísticos, 213=Profesionales en ciencias biológicas, 214=Ingenieros ( excluyendo electrotecnólogos), 215=Ingenieros en electrotecnología, 216=Arquitectos, urbanistas, agrimensores y diseñadores, 221=Médicos, 222=Profesionales de enfermería y partería, 223=Profesionales de medicina tradicional y alternativa, 224=Practicantes paramédicos, 225=Practicantes, paramédicos, 226=Otros profesionales de la salud, 231=Profesores de universidades y otros establecimientos de la enseñanza superior, 232=Profesores de formación profesional, 233=Profesores de enseñanza secundaria, 234=Maestros de enseñanza primaria y maestros preescolares, 235=Otros profesionales de la enseñanza, 241=Especialistas en finanzas, 242=Especialistas en organización de administración, 243=Profesionales de las ventas, la comercialización y las relaciones públicas, 251=Desarrolladores y analistas de software y multimedia, 252=Especialistas en bases de datos y en redes de computadores, 261=Profesionales en derecho, 262=Archivistas, bibliotecarios, curadores y afines, 263=Especialistas en ciencias sociales y teología
264=Autores, periodistas y lingüistas, 265=Artistas creativos e interpretativos

Técnicos y profesionales de nivel medio: 311=Técnicos en ciencias físicas y en ingeniería, 312=Supervisores en ingeniería de minas, de industrias manufactureras y de la construcción, 313=Técnicos en control de procesos, 314=Técnicos y profesionales de nivel medio en ciencias biológicas y afines, 315=Técnicos y controladores en navegación marítima y aeronáutica, 400=Personal de apoyo administrativo, 500=Trabajadores de los servicios y vendedores de comercios y mercados, 600=Agricultores y trabajadores calificados agropecuarios y pesqueros
700=Oficiales, operarios y artesanos de artes mecánicas y de otros oficios, 800=Operadores de instalaciones y máquinas y ensambladores, 900=Ocupaciones elementales, 000=Ocupaciones militares

In [423]:
df_doct['CODALP'].sort_values().unique() # Blank as 'NNN'

array(['000', '100', '211', '212', '213', '214', '215', '216', '221',
       '222', '223', '224', '225', '226', '231', '232', '233', '234',
       '235', '241', '242', '243', '251', '252', '261', '262', '263',
       '264', '265', '311', '313', '314', '315', '400', '500', '600',
       '900', 'NNN'], dtype=object)

In [424]:
re_sitlab('CODALP',blank='NNN')

['000' '100' '211' '212' '213' '214' '215' '216' '221' '222' '223' '224'
 '225' '226' '231' '232' '233' '234' '235' '241' '242' '243' '251' '252'
 '261' '262' '263' '264' '265' '311' '313' '314' '315' '400' '500' '600'
 '900' 'NNN']
                                                                                                                                                                                           CODALP
                                                                                                                                                                                           unique
SITLAB                                                                                                                                                                                           
1       [226, 231, 221, 252, 213, 211, 216, 223, 233, 214, 261, 235, 263, 262, 212, 100, 900, 400, 242, 243, 225, 232, 314, 234, 241, 313, 264, 222, 500, 251, 311, 265, 215, 315, 000, 6

#### 'CATEGPROF'
(Solo para respuesta 231 en variable 'CODALP')  
Categoría profesional o puesto de trabajo que ocupaba el 31 de diciembre de 2009: 1=Catedrático, 2=Profesor titular, 3=Profesor asociado, emerito, visitante, ayudante o similar, 4=Otro.

In [425]:
df_doct['CATEGPROF'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4'], dtype=object)

In [426]:
df_doct[['CODALP','CATEGPROF']].groupby('CODALP').agg(['unique']) # No missing values

,CATEGPROF
,unique
CODALP,
0.0,[ ]
100.0,[ ]
211.0,[ ]
212.0,[ ]
213.0,[ ]
214.0,[ ]
215.0,[ ]
216.0,[ ]


In [427]:
df_doct['CATEGPROF'].replace({' ':np.nan},inplace=True) # None for 'CODALP' different from 231
df_doct['CATEGPROF']=df_doct['CATEGPROF'].astype('float')
df_doct['CATEGPROF'].sort_values().unique()

array([ 1.,  2.,  3.,  4., nan])

#### 'HLECT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Proporción de su trabajo dedicado a actividades de docencia: 1=Nada, 2=Menos del 25 %; 3=Del 25 a 50 %; 4=Del 50 al 75 %; 5=Más del 75%. 

In [430]:
df_doct['HLECT']=df_raw['HLECT']
df_doct['HLECT'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4', '5'], dtype=object)

In [431]:
re_sitlab('HLECT',blank=' ')

[' ' '1' '2' '3' '4' '5']
                  HLECT
                 unique
SITLAB                 
1       [1, 4, 3, 2, 5]
2                   [ ]
3                   [ ]
[ 1.  2.  3.  4.  5. nan]


#### 'TIPOCONT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Tipo de contrato al que estaba sujeto: 1=Indefinido, 2=Temporal. 

In [434]:
df_doct['TIPOCONT'].sort_values().unique() # Blank as ' '

array([' ', '1', '2'], dtype=object)

In [435]:
df_doct[['SITLAB','TIPOCONT','AUTON']].groupby(['SITLAB','AUTON']).agg(['unique']) #  No missing values

TIPOCONT
               unique
SITLAB AUTON         
1      0       [1, 2]
       1          [ ]
2      0          [ ]
3      0          [ ]

In [436]:
df_doct['TIPOCONT'].replace({' ':None},inplace=True) # None for 'SITLAB' different from 1 and 'AUTON' different from 0
df_doct['TIPOCONT']=df_doct['TIPOCONT'].astype('float')
df_doct['TIPOCONT'].sort_values().unique()

array([ 1.,  2., nan])

#### 'JORLAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Tipo de jornada laboral: 1=A tiempo completo, 2=A tiempo parcial.

In [440]:
df_doct['JORLAB'].sort_values().unique() # Blank as ' '

array([' ', '1', '2'], dtype=object)

In [441]:
df_doct[['SITLAB','JORLAB']].groupby('SITLAB').agg(['unique']) # No missing values

,JORLAB
,unique
SITLAB,
1,"[1, 2]"
2,[ ]
3,[ ]


In [442]:
df_doct['JORLAB'].replace({' ':None},inplace=True) # None for 'SITLAB' different from 1 
df_doct['JORLAB']=df_doct['JORLAB'].astype('float')
df_doct['JORLAB'].sort_values().unique()

array([ 1.,  2., nan])

#### 'BUSCATRAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Tiempo parcial' (2) en variable 'JORLAB')  
¿Estaba usted buscando un trabajo a tiempo completo en diciembre de 2009? 0=No, 1=Sí.

In [297]:
df_doct['BUSCATRAB'].sort_values().unique() # Blank as ' '

array([' ', '0', '1'], dtype=object)

In [298]:
df_doct[['SITLAB','JORLAB','BUSCATRAB']].groupby(['SITLAB','JORLAB']).agg(['unique']) #  No missing values

BUSCATRAB
                 unique
SITLAB JORLAB          
1      1.0          [ ]
       2.0       [1, 0]

In [299]:
df_doct['BUSCATRAB'].replace({' ':np.nan},inplace=True) # None for 'SITLAB' different from 1 and 'JORLAB' different from 2
df_doct['BUSCATRAB']=df_doct['BUSCATRAB'].astype('float')
df_doct['BUSCATRAB'].sort_values().unique()

array([ 0.,  1., nan])

#### 'NIVELMIN'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel mínimo requerido para el puesto que tenía usted en su trabajo principal el 31 de diciembre de 2009: 1=Postdoctorado, 2=Doctor universitario, 3=Licenciado, arquitecto, ingeniero, 4=Diplomado, arquitecto técnico, ingeniero técnico, 5=Ciclos formativos de grado superior, 6=Ciclos formativos de grado medio,bachiller, 7= Otros.

In [444]:
df_doct['NIVELMIN'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [445]:
re_sitlab('NIVELMIN',blank=' ')

[' ' '1' '2' '3' '4' '5' '6' '7']
                     NIVELMIN
                       unique
SITLAB                       
1       [3, 2, 1, 4, 6, 7, 5]
2                         [ ]
3                         [ ]
[ 1.  2.  3.  4.  5.  6.  7. nan]


#### 'NIVELEST'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel que considera que es el adecuado para el puesto que tenía usted en su trabajo principal el 31 de
diciembre de 2009 (Códigos: véase la variable 'NIVELMIN').

In [446]:
df_doct['NIVELEST'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [447]:
re_sitlab('NIVELEST',blank=' ')

[' ' '1' '2' '3' '4' '5' '6' '7']
                     NIVELEST
                       unique
SITLAB                       
1       [3, 2, 1, 7, 5, 4, 6]
2                         [ ]
3                         [ ]
[ 1.  2.  3.  4.  5.  6.  7. nan]


#### 'RELTRABDOC'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Grado de relación entre el trabajo principal que desempeñaba a 31 de diciembre de 2009 y sus estudios
de doctorado:1=Alto, 2=Medio, 3=Bajo.

In [448]:
df_doct['RELTRABDOC'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3'], dtype=object)

In [449]:
re_sitlab('RELTRABDOC',blank=' ')

[' ' '1' '2' '3']
       RELTRABDOC
           unique
SITLAB           
1       [3, 1, 2]
2             [ ]
3             [ ]
[ 1.  2.  3. nan]


#### 'NIVELSAT1'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su salario el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [455]:
df_doct['NIVELSAT1'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4'], dtype=object)

In [456]:
re_sitlab('NIVELSAT1',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT1
              unique
SITLAB              
1       [1, 2, 4, 3]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con sus beneficios económicos el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [457]:
re_sitlab('NIVELSAT2',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT2
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT3'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su estabilidad laboral el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [459]:
re_sitlab('NIVELSAT3',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT3
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT4'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su localización laboral el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [460]:
re_sitlab('NIVELSAT4',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT4
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT5'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con sus condiciones laborales el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [461]:
re_sitlab('NIVELSAT5',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT5
              unique
SITLAB              
1       [1, 2, 4, 3]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT6'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con sus oportunidades para promocionar el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [462]:
re_sitlab('NIVELSAT6',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT6
              unique
SITLAB              
1       [4, 2, 1, 3]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT7'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con el componente o reto intelectual el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [463]:
re_sitlab('NIVELSAT7',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT7
              unique
SITLAB              
1       [3, 1, 2, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT8'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su nivel de responsabilidad el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [464]:
re_sitlab('NIVELSAT8',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT8
              unique
SITLAB              
1       [2, 1, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT9'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su grado de independencia el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [465]:
re_sitlab('NIVELSAT9',blank=' ')

[' ' '1' '2' '3' '4']
           NIVELSAT9
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT10'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su contribución a la sociedad el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [467]:
re_sitlab('NIVELSAT10',blank=' ')

[' ' '1' '2' '3' '4']
          NIVELSAT10
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT11'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con su estatus social el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [468]:
re_sitlab('NIVELSAT11',blank=' ')

[' ' '1' '2' '3' '4']
          NIVELSAT11
              unique
SITLAB              
1       [1, 2, 3, 4]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT12'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de satisfacción con sus conciliación horario laboral/actividades personales el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [469]:
re_sitlab('NIVELSAT12',blank=' ')

[' ' '1' '2' '3' '4']
          NIVELSAT12
              unique
SITLAB              
1       [1, 2, 4, 3]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'NIVELSAT13'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Evaluación global de satisfacción el 31 de diciembre de 2009: 1=Alta, 2=Media, 3=Baja, 4=Ninguna.

In [470]:
re_sitlab('NIVELSAT13',blank=' ')

[' ' '1' '2' '3' '4']
          NIVELSAT13
              unique
SITLAB              
1       [1, 2, 4, 3]
2                [ ]
3                [ ]
[ 1.  2.  3.  4. nan]


#### 'ACTIANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
En los últimos 5 años, ¿ha desempeñado alguna actividad laboral distinta de la que desarrollaba a 31 de diciembre
de 2009? 0=No, 1=Sí.

In [481]:
df_doct['ACTIANT']=df_raw['ACTIANT']
df_doct['ACTIANT'].sort_values().unique() # Blank as ' '

array([' ', '0', '1'], dtype=object)

In [482]:
re_sitlab('ACTIANT',blank=' ')

[' ' '0' '1']
       ACTIANT
        unique
SITLAB        
1       [0, 1]
2          [ ]
3          [ ]
[ 0.  1. nan]


#### 'SECTORANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  
Sector al que pertenece la organización en la que trabajaba (Códigos: véase la variable 'SECTOR')

In [486]:
df_doct['SECTORANT'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4'], dtype=object)

In [487]:
def re_actiant(col,blank,value=None,tipus='float'):
    print (df_doct[col].sort_values().unique())
    print (df_doct[['SITLAB','ACTIANT',col]].groupby(['SITLAB','ACTIANT']).agg(['unique']))
    df_doct[col].replace({blank:value},inplace=True) 
    df_doct[col]=df_doct[col].astype(tipus)
    print (df_doct[col].sort_values().unique())
    return 

In [488]:
re_actiant('SECTORANT',blank=' ')

[' ' '1' '2' '3' '4']
                   SECTORANT
                      unique
SITLAB ACTIANT              
1      0.0               [ ]
       1.0      [2, 1, 3, 4]
[ 1.  2.  3.  4. nan]


#### 'ACTIPRIANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  
Actividad laboral que desarrollaba en su ocupación anterior de acuerdo con la Clasificación internacional uniforme de ocupaciones (ISCO-08) (Códigos: véase la variable 'CODALP').

In [493]:
df_doct['ACTIPRIANT'].sort_values().unique() # Blank as 'NNN'

array(['100', '211', '212', '213', '214', '215', '216', '221', '223',
       '224', '225', '226', '231', '232', '233', '235', '241', '242',
       '243', '251', '252', '261', '262', '263', '264', '265', '311',
       '313', '314', '400', '500', '700', '800', '900', 'NNN'],
      dtype=object)

In [494]:
re_actiant('ACTIPRIANT',blank='NNN')

['100' '211' '212' '213' '214' '215' '216' '221' '223' '224' '225' '226'
 '231' '232' '233' '235' '241' '242' '243' '251' '252' '261' '262' '263'
 '264' '265' '311' '313' '314' '400' '500' '700' '800' '900' 'NNN']
                                                                                                                                                                                ACTIPRIANT
                                                                                                                                                                                    unique
SITLAB ACTIANT                                                                                                                                                                            
1      0.0                                                                                                                                                                           [NNN]
       1.0      [226, 214, 211, 231, 2

#### 'ACTIANTANO'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  
Duración de su ocupación anterior: años completos.

In [495]:
df_doct['ACTIANTANO'].sort_values().unique() # Blank as ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '30', '31', '33',
       '34', '39'], dtype=object)

In [496]:
re_actiant('ACTIANTANO',blank=' .')

[' .' ' 0' ' 1' ' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '10' '11' '12'
 '13' '14' '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26'
 '27' '28' '30' '31' '33' '34' '39']
                                                                                                                                              ACTIANTANO
                                                                                                                                                  unique
SITLAB ACTIANT                                                                                                                                          
1      0.0                                                                                                                                          [ .]
       1.0      [ 3,  2, 14,  4,  1, 19,  6, 20,  5, 15,  0,  7, 11, 12,  9, 18, 10,  8, 34, 16, 27, 39, 17, 24, 25, 30, 13, 28, 23, 21, 22, 33, 31, 26]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.

#### 'ACTIANTMES'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  
Duración de su ocupación anterior: meses que exceden los años declarados en 'ACTIANTANO'.

In [497]:
df_doct['ACTIANTMES'].sort_values().unique() # Blank as ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '11', '12'], dtype=object)

In [498]:
re_actiant('ACTIANTMES',blank=' .')

[' .' ' 0' ' 1' ' 2' ' 3' ' 4' ' 5' ' 6' ' 7' ' 8' ' 9' '10' '11' '12']
                                                          ACTIANTMES
                                                              unique
SITLAB ACTIANT                                                      
1      0.0                                                      [ .]
       1.0      [ 2,  0,  9,  6,  4,  7,  1,  3, 10, 12, 11,  5,  8]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. nan]


In [ ]:
# New variable (not in original dataset):
df_doct['Temp_act_ant']=df_doct['ACTIANTANO']+df_doct['ACTIANTMES']/12
df_doct['Temp_act_ant'].sort_values().unique() # VALORES NULOS. ¿DESCARTAR?

#### 'TRABPOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
¿Era un puesto postdoctoral su principal empleo el 31 de diciembre de 2009? 0=No, 1=Sí. 

In [499]:
df_doct['TRABPOS'].sort_values().unique() # Blank as ' '

array([' ', '0', '1'], dtype=object)

In [501]:
re_sitlab('TRABPOS',blank=' ')

[' ' '0' '1']
       TRABPOS
        unique
SITLAB        
1       [0, 1]
2          [ ]
3          [ ]
[ 0.  1. nan]


#### 'MOTIVOPOS1'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era realizar prácticas en su campo de estudio una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [508]:
df_doct['MOTIVOPOS1'].sort_values().unique() # Blank as ' '

array([' ', '0', '1'], dtype=object)

In [518]:
def re_trabpos(col,blank=' ',value=None,tipus='float'):
    print (df_doct[col].sort_values().unique())
    print (df_doct[['SITLAB','TRABPOS',col]].groupby(['SITLAB','TRABPOS']).agg(['unique']))
    df_doct[col].replace({blank:value},inplace=True) 
    df_doct[col]=df_doct[col].astype(tipus)
    print (df_doct[col].sort_values().unique())
    return 

In [509]:
re_trabpos('MOTIVOPOS1')

[' ' '0' '1']
               MOTIVOPOS1
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [1, 0]
[ 0.  1. nan]


#### 'MOTIVOPOS2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era realizar prácticas en un campo diferente al suyo una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [511]:
re_trabpos('MOTIVOPOS2')

[' ' '0' '1']
               MOTIVOPOS2
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [1, 0]
[ 0.  1. nan]


#### 'MOTIVOPOS3'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era realizar investigación de forma independiente una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [512]:
re_trabpos('MOTIVOPOS3')

[' ' '0' '1']
               MOTIVOPOS3
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [1, 0]
[ 0.  1. nan]


#### 'MOTIVOPOS4'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era **trabajar con una persona específica o en un lugar concreto** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [513]:
re_trabpos('MOTIVOPOS4')

[' ' '0' '1']
               MOTIVOPOS4
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [0, 1]
[ 0.  1. nan]


#### 'MOTIVOPOS5'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era **llevar a cabo actividades docentes** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [514]:
re_trabpos('MOTIVOPOS5')

[' ' '0' '1']
               MOTIVOPOS5
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [0, 1]
[ 0.  1. nan]


#### 'MOTIVOPOS6'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era **no haber podido elegir otro empleo** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [515]:
re_trabpos('MOTIVOPOS6')

[' ' '0' '1']
               MOTIVOPOS6
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [0, 1]
[ 0.  1. nan]


#### 'MOTIVOPOS7'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Era **que es el trabajo que se suele realizar en su campo de estudio** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [516]:
re_trabpos('MOTIVOPOS7')

[' ' '0' '1']
               MOTIVOPOS7
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [1, 0]
[ 0.  1. nan]


#### 'MOTIVOPOS8'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
¿Eran **otras razones** las que le llevaron a realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [517]:
re_trabpos('MOTIVOPOS8')

[' ' '0' '1']
               MOTIVOPOS8
                   unique
SITLAB TRABPOS           
1      0.0            [ ]
       1.0         [0, 1]
[ 0.  1. nan]


#### 'PORCINVES'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
Porcentaje de su tiempo como postdoctorado empleado en **actividades de investigación**.

In [524]:
df_doct['PORCINVES'].sort_values().unique() # Blank as '  .'

array(['  .', '  0', '  5', ' 10', ' 15', ' 20', ' 25', ' 30', ' 34',
       ' 35', ' 40', ' 45', ' 49', ' 50', ' 55', ' 60', ' 65', ' 70',
       ' 75', ' 80', ' 85', ' 90', ' 95', ' 98', ' 99', '100'],
      dtype=object)

In [525]:
re_trabpos('PORCINVES',blank='  .')

['  .' '  0' '  5' ' 10' ' 15' ' 20' ' 25' ' 30' ' 34' ' 35' ' 40' ' 45'
 ' 49' ' 50' ' 55' ' 60' ' 65' ' 70' ' 75' ' 80' ' 85' ' 90' ' 95' ' 98'
 ' 99' '100']
                                                                                                                                    PORCINVES
                                                                                                                                       unique
SITLAB TRABPOS                                                                                                                               
1      0.0                                                                                                                              [  .]
       1.0      [100,  95,  50,  40,  25,  98,  75,  80,  90,  60,  20,  30,  45,  85,  70,   0,  10,   5,  35,  34,  15,  99,  55,  49,  65]
[  0.   5.  10.  15.  20.  25.  30.  34.  35.  40.  45.  49.  50.  55.
  60.  65.  70.  75.  80.  85.  90.  95.  98.  99. 100.  na

#### 'PORCDOC'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
Porcentaje de su tiempo como postdoctorado empleado en **actividades de docencia**.

In [526]:
df_doct['PORCDOC'].sort_values().unique() # Blank as '  .'

array(['  .', '  0', '  1', '  2', '  5', '  8', ' 10', ' 15', ' 20',
       ' 25', ' 30', ' 33', ' 35', ' 40', ' 45', ' 49', ' 50', ' 55',
       ' 60', ' 65', ' 70', ' 75', ' 80', ' 85', ' 90', '100'],
      dtype=object)

In [527]:
re_trabpos('PORCDOC',blank='  .')

['  .' '  0' '  1' '  2' '  5' '  8' ' 10' ' 15' ' 20' ' 25' ' 30' ' 33'
 ' 35' ' 40' ' 45' ' 49' ' 50' ' 55' ' 60' ' 65' ' 70' ' 75' ' 80' ' 85'
 ' 90' '100']
                                                                                                                                      PORCDOC
                                                                                                                                       unique
SITLAB TRABPOS                                                                                                                               
1      0.0                                                                                                                              [  .]
       1.0      [  0,   5,  50,  60,  75,   2,  25,  20,  10,  30,  80,  70,  55,  15,  40, 100,  33,   1,  35,  45,  85,  90,  49,  65,   8]
[  0.   1.   2.   5.   8.  10.  15.  20.  25.  30.  33.  35.  40.  45.
  49.  50.  55.  60.  65.  70.  75.  80.  85.  90. 100.  na

#### 'PORCOTRAS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
Porcentaje de su tiempo como postdoctorado empleado en **otras actividades**.

In [528]:
df_doct['PORCOTRAS'].sort_values().unique() # Blank as '  .'

array(['  .', '  0', '  2', '  5', ' 10', ' 15', ' 20', ' 25', ' 30',
       ' 33', ' 35', ' 40', ' 50', ' 55', ' 60', ' 65', ' 70', ' 75',
       ' 78', ' 80', ' 85', ' 90', '100'], dtype=object)

In [529]:
re_trabpos('PORCOTRAS',blank='  .')

['  .' '  0' '  2' '  5' ' 10' ' 15' ' 20' ' 25' ' 30' ' 33' ' 35' ' 40'
 ' 50' ' 55' ' 60' ' 65' ' 70' ' 75' ' 78' ' 80' ' 85' ' 90' '100']
                                                                                                                     PORCOTRAS
                                                                                                                        unique
SITLAB TRABPOS                                                                                                                
1      0.0                                                                                                               [  .]
       1.0      [  0,  25,  10, 100,   5,  30,  40,  75,  20,  50,  80,  33,  85,  60,  15,  70,  65,  78,  55,   2,  90,  35]
[  0.   2.   5.  10.  15.  20.  25.  30.  33.  35.  40.  50.  55.  60.
  65.  70.  75.  78.  80.  85.  90. 100.  nan]


In [530]:
# Variable temporal para comprobacion:
df_doct['Temp_porc']=df_doct['PORCINVES']+df_doct['PORCDOC']+df_doct['PORCOTRAS']
df_doct['Temp_porc'].sort_values().unique()

array([100.,  nan])

In [531]:
del df_doct['Temp_porc']

#### 'FINANCPOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  
Fuente principal de **financiación** de ese postdoctorado: 1=Empresas, 2=AAPP, 3=Centros de enseñanza superior, 4=IPSFL, 5=Fundaciones privadas, 6=OTRAS.

In [534]:
df_doct['FINANPOS'].sort_values().unique() # Blank as ' '

array([' ', '1', '2', '3', '4', '5', '6'], dtype=object)

In [535]:
re_trabpos('FINANPOS')

[' ' '1' '2' '3' '4' '5' '6']
                          FINANPOS
                            unique
SITLAB TRABPOS                    
1      0.0                     [ ]
       1.0      [2, 3, 5, 6, 1, 4]
[ 1.  2.  3.  4.  5.  6. nan]


#### 'GESBUSEMP'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB')  
Durante enero de 2010, ¿hizo alguna gestión para buscar empleo? 0=No, 1=Sí.

In [536]:
df_doct['GESBUSEMP'].sort_values().unique() # Blank as ' '

array([' ', '0', '1'], dtype=object)

In [537]:
re_sitlab('GESBUSEMP',blank=' ')

[' ' '0' '1']
       GESBUSEMP
          unique
SITLAB          
1            [ ]
2         [1, 0]
3         [0, 1]
[ 0.  1. nan]


#### 'PASOSEMP1'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Se puso en contacto con una **empresa de trabajo temporal**? 1=Sí, blanco.

In [554]:
df_doct['PASOSEMP1'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [555]:
df_doct[['SITLAB','GESBUSEMP','PASOSEMP1']].groupby(['SITLAB','GESBUSEMP']).agg(['unique']) 

PASOSEMP1
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]

In [556]:
def pasosemp(col):
    print (df_doct[col].sort_values().unique())
    print (df_doct[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP']).agg(['unique']))
    # Se imputa respuesta 'No' (0) a quien no contesta sí habiendo sido preguntado:
    df_doct[col].mask((df_doct['GESBUSEMP']==1)&(df_doct[col]==' '),0,inplace=True) 
    # Se imputa None cuando no se formula la pregunta:
    df_doct[col].where((df_doct[col]==0)|(df_doct[col]=='1'),None,inplace=True)
    df_doct[col]=df_doct[col].astype('float')
    print (df_doct[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP']).agg(['unique']))
    return 

In [557]:
pasosemp('PASOSEMP1')

[' ' '1']
                 PASOSEMP1
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP1
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP2'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Buscó en **oficinas de empleo**? 1=Sí, blanco.

In [553]:
pasosemp('PASOSEMP2')

[' ' '1']
                 PASOSEMP2
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [1,  ]
                   PASOSEMP2
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [1.0, 0.0]


#### 'PASOSEMP3'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Se puso en contacto directamente con las **empresas** que necesitan personal? 1=Sí, blanco.

In [558]:
pasosemp('PASOSEMP3')

[' ' '1']
                 PASOSEMP3
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP3
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP4'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Revisó y respondió **anuncios** de periódicos? 1=Sí, blanco.

In [559]:
pasosemp('PASOSEMP4')

[' ' '1']
                 PASOSEMP4
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP4
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP5'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Buscó en **Internet**? 1=Sí, blanco.

In [560]:
pasosemp('PASOSEMP5')

[' ' '1']
                 PASOSEMP5
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP5
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP6'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Buscó ayuda entre **amigos y parientes**? 1=Sí, blanco.

In [561]:
pasosemp('PASOSEMP6')

[' ' '1']
                 PASOSEMP6
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [ , 1]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP6
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [0.0, 1.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP7'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Buscó ayuda entre **profesionales conocidos del sector**? 1=Sí, blanco.

In [562]:
pasosemp('PASOSEMP7')

[' ' '1']
                 PASOSEMP7
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP7
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP8'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Buscacó terrenos, maquinaria o equipamiento para establecer **su propia empresa**? 1=Sí, blanco.

In [563]:
pasosemp('PASOSEMP8')

[' ' '1']
                 PASOSEMP8
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [ , 1]
3      0.0             [ ]
       1.0             [ ]
                   PASOSEMP8
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [0.0, 1.0]
3      0.0             [nan]
       1.0             [0.0]


#### 'PASOSEMP9'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Hizo **entrevistas de trabajo**? 1=Sí, blanco.

In [565]:
pasosemp('PASOSEMP9')

[' ' '1']
                 PASOSEMP9
                    unique
SITLAB GESBUSEMP          
2      0.0             [ ]
       1.0          [1,  ]
3      0.0             [ ]
       1.0          [ , 1]
                   PASOSEMP9
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [1.0, 0.0]
3      0.0             [nan]
       1.0        [0.0, 1.0]


#### 'PASOSEMP10'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Solicitó **permisos, licencias o recursos financieros**? 1=Sí, blanco.

In [564]:
pasosemp('PASOSEMP10')

[' ' '1']
                 PASOSEMP10
                     unique
SITLAB GESBUSEMP           
2      0.0              [ ]
       1.0           [ , 1]
3      0.0              [ ]
       1.0              [ ]
                  PASOSEMP10
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [0.0, 1.0]
3      0.0             [nan]
       1.0             [0.0]


#### 'PASOSEMP11'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  
¿Realizó **otras** gestiones? 1=Sí, blanco.

In [566]:
pasosemp('PASOSEMP11')

[' ' '1']
                 PASOSEMP11
                     unique
SITLAB GESBUSEMP           
2      0.0              [ ]
       1.0           [ , 1]
3      0.0              [ ]
       1.0           [1,  ]
                  PASOSEMP11
                      unique
SITLAB GESBUSEMP            
2      0.0             [nan]
       1.0        [0.0, 1.0]
3      0.0             [nan]
       1.0        [1.0, 0.0]


#### 'NOPASOSEMP1'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿La **enfermedad o discapacidad** fueron una razón para no buscar empleo? 1=Sí, blanco.

In [567]:
df_doct['NOPASOSEMP1'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [568]:
df_doct[['SITLAB','GESBUSEMP','NOPASOSEMP1']].groupby(['SITLAB','GESBUSEMP']).agg(['unique']) 

NOPASOSEMP1
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]

In [570]:
def nopasosemp(col):
    print (df_doct[col].sort_values().unique())
    print (df_doct[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP']).agg(['unique']))
    # Se imputa respuesta 'No' (0) a quien no contesta sí habiendo sido preguntado:
    df_doct[col].mask((df_doct['GESBUSEMP']==0)&(df_doct[col]==' '),0,inplace=True) 
    # Se imputa None cuando no se formula la pregunta:
    df_doct[col].where((df_doct[col]==0)|(df_doct[col]=='1'),None,inplace=True)
    df_doct[col]=df_doct[col].astype('float')
    print (df_doct[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP']).agg(['unique']))
    return 

In [571]:
nopasosemp('NOPASOSEMP1')

[' ' '1']
                 NOPASOSEMP1
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP1
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP2'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿El **cuidado de hijos** fue una razón para no buscar empleo? 1=Sí, blanco.

In [572]:
nopasosemp('NOPASOSEMP2')

[' ' '1']
                 NOPASOSEMP2
                      unique
SITLAB GESBUSEMP            
2      0.0            [1,  ]
       1.0               [ ]
3      0.0            [1,  ]
       1.0               [ ]
                 NOPASOSEMP2
                      unique
SITLAB GESBUSEMP            
2      0.0        [1.0, 0.0]
       1.0             [nan]
3      0.0        [1.0, 0.0]
       1.0             [nan]


#### 'NOPASOSEMP3'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿El **cuidado de otras personas dependientes** fue una razón para no buscar empleo? 1=Sí, blanco.

In [573]:
nopasosemp('NOPASOSEMP3')

[' ' '1']
                 NOPASOSEMP3
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP3
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP4'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Fueron **otras responsabilidades familiares** una razón para no buscar empleo? 1=Sí, blanco.

In [574]:
nopasosemp('NOPASOSEMP4')

[' ' '1']
                 NOPASOSEMP4
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP4
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP5'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Fue la **jubilación** una razón para no buscar empleo? 1=Sí, blanco.

In [575]:
nopasosemp('NOPASOSEMP5')

[' ' '1']
                 NOPASOSEMP5
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP5
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP6'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Fue la **asistencia a instituciones educativas** una razón para no buscar empleo? 1=Sí, blanco.

In [576]:
nopasosemp('NOPASOSEMP6')

[' ' '1']
                 NOPASOSEMP6
                      unique
SITLAB GESBUSEMP            
2      0.0               [ ]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP6
                      unique
SITLAB GESBUSEMP            
2      0.0             [0.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP7'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Fue la **convicción** de que no existe ningún trabajo disponible para usted una razón para no buscar empleo? 1=Sí, blanco.

In [577]:
nopasosemp('NOPASOSEMP7')

[' ' '1']
                 NOPASOSEMP7
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP7
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP8'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Fue que **no necesita o no quiere trabajar** una razón para no buscar empleo? 1=Sí, blanco.

In [578]:
nopasosemp('NOPASOSEMP8')

[' ' '1']
                 NOPASOSEMP8
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP8
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'NOPASOSEMP9'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  
¿Tenía **otros** motivos para no buscar empleo? 1=Sí, blanco.

In [579]:
nopasosemp('NOPASOSEMP9')

[' ' '1']
                 NOPASOSEMP9
                      unique
SITLAB GESBUSEMP            
2      0.0            [ , 1]
       1.0               [ ]
3      0.0            [ , 1]
       1.0               [ ]
                 NOPASOSEMP9
                      unique
SITLAB GESBUSEMP            
2      0.0        [0.0, 1.0]
       1.0             [nan]
3      0.0        [0.0, 1.0]
       1.0             [nan]


#### 'CONT1'
Continente del primer país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [584]:
df_doct['CONT1'].sort_values().unique()

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'NMOV', 'OCEA', 'RAME', 'REUR'],
      dtype=object)

#### 'ANOIPAIS1'
Año de inicio del tiempo de residencia en el primer país mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [585]:
df_doct['ANOIPAIS1'].sort_values().unique() # Blank as '    .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009'], dtype=object)

In [593]:
def re_cont(col,cont,blank,tipus='float'):
    print (df_doct[[col,cont]].groupby(cont).agg(['unique']))
    df_doct[col].replace({blank:None},inplace=True)
    df_doct[col]=df_doct[col].astype(tipus)
    print (df_doct[[col,cont]].groupby(cont).agg(['unique']))
    return

In [594]:
re_cont('ANOIPAIS1','CONT1',blank='   .')

                                                          ANOIPAIS1
                                                             unique
CONT1                                                              
AFRI                                                   [2009, 2007]
ASIA                                                   [2009, 2008]
EU27   [2006, 2008, 2009, 2003, 2002, 2005, 2007, 2001, 2000, 2004]
EUCA               [2000, 2008, 2009, 2005, 2004, 2006, 2002, 2007]
NMOV                                                         [None]
OCEA                                                         [2007]
RAME                                                   [2009, 2006]
REUR                                 [2009, 2008, 2002, 2007, 2004]
                                                                              ANOIPAIS1
                                                                                 unique
CONT1                                                                       

#### 'ANOFPAIS1'
Año final del tiempo de residencia en el primer país mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [595]:
df_doct['ANOFPAIS1'].sort_values().unique() # Blank as '   .'

array(['   .', '2009'], dtype=object)

In [596]:
re_cont('ANOFPAIS1','CONT1',blank='   .')
# Los registros de duración de la residencia en España han sido suprimidos

      ANOFPAIS1
         unique
CONT1          
AFRI     [2009]
ASIA     [2009]
EU27     [2009]
EUCA     [2009]
NMOV     [   .]
OCEA     [2009]
RAME     [2009]
REUR     [2009]
      ANOFPAIS1
         unique
CONT1          
AFRI   [2009.0]
ASIA   [2009.0]
EU27   [2009.0]
EUCA   [2009.0]
NMOV      [nan]
OCEA   [2009.0]
RAME   [2009.0]
REUR   [2009.0]


#### 'MESES1'
Duración en meses del tiempo de residencia en el primer país mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [597]:
df_doct['MESES1'].sort_values().unique() # Blank as '  .'

array(['  .', '  2', '  3', '  4', '  5', '  6', '  7', '  8', '  9',
       ' 10', ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18',
       ' 19', ' 20', ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27',
       ' 28', ' 29', ' 30', ' 31', ' 32', ' 33', ' 34', ' 35', ' 36',
       ' 37', ' 38', ' 39', ' 40', ' 41', ' 42', ' 43', ' 44', ' 45',
       ' 46', ' 47', ' 48', ' 49', ' 50', ' 51', ' 52', ' 53', ' 54',
       ' 55', ' 56', ' 57', ' 58', ' 59', ' 60', ' 61', ' 62', ' 63',
       ' 64', ' 65', ' 66', ' 67', ' 68', ' 69', ' 70', ' 71', ' 72',
       ' 73', ' 74', ' 75', ' 76', ' 77', ' 78', ' 79', ' 80', ' 81',
       ' 82', ' 83', ' 84', ' 85', ' 86', ' 87', ' 88', ' 89', ' 90',
       ' 91', ' 92', ' 93', ' 94', ' 95', ' 96', ' 97', ' 98', ' 99',
       '100', '101', '102', '104', '105', '106', '107', '108', '109',
       '110', '111', '112', '113', '114', '115', '116', '117', '120'],
      dtype=object)

In [599]:
re_cont('MESES1','CONT1',blank='  .')

                                                                                                                                                                                                                                                                                                            MESES1
                                                                                                                                                                                                                                                                                                            unique
CONT1                                                                                                                                                                                                                                                                                                             
AFRI                                                                           

#### 'CONT2'
Continente del segundo país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [600]:
df_doct['CONT2'].sort_values().unique() # Blank as '   .'

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'NMOV', 'OCEA', 'RAME', 'REUR'],
      dtype=object)

#### 'ANOIPAIS2'
Año de inicio del tiempo de residencia en el segundo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [669]:
df_doct['ANOIPAIS2'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009'], dtype=object)

In [670]:
re_cont('ANOIPAIS2','CONT2',blank='   .')

                                                          ANOIPAIS2
                                                             unique
CONT2                                                              
AFRI                                 [2002, 2003, 2004, 2007, 2000]
ASIA                     [2007, 2008, 2009, 2003, 2000, 2002, 2004]
EU27   [2005, 2000, 2007, 2001, 2002, 2009, 2008, 2003, 2004, 2006]
EUCA   [2006, 2000, 2004, 2001, 2005, 2002, 2007, 2008, 2003, 2009]
NMOV                                                   [   ., 2006]
OCEA                     [2004, 2009, 2003, 2005, 2000, 2007, 2001]
RAME   [2006, 2008, 2003, 2001, 2007, 2000, 2004, 2009, 2002, 2005]
REUR         [2008, 2009, 2005, 2007, 2001, 2000, 2002, 2006, 2003]
                                                                              ANOIPAIS2
                                                                                 unique
CONT2                                                                       

In [671]:
df_doct[(df_doct['CONT2']=='NMOV')&(df_doct['ANOIPAIS2'].notna())] 
# Observación peculiar: periodo intermedio en España con registros de duración no suprimidos

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,Tmpo_doc,Tmpo_s/T,"(3416, ANOFPAIS2)","(733, ANOFPAIS2)"
3416,410103,1,1960,01,EU27,1,EU27,None,None,2,4,0,0,1,8.0,3.0,1,5,2,NaN,1,1,1,1,01,10.0,01,2,None,0,508,1,1,1,2004,7,6,0,EU27,1983,5,1,0.0,0.0,1,90,4,0,85.0,2009.0,1.0,EU27,3.0,231.0,4.0,4.0,1.0,1.0,NaN,2.0,2.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,2.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0,265.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,60.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EU27,2006.0,2009.0,42.0,NMOV,2006.0,2006.0,5.0,EU27,2000.0,2006.0,73.0,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,,,,1,,,,,,1,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,1,,,1,,1,,60,7,4,0,0,0,0,0,0,1,1,1,25.66800106,6.0,0.0,NaN,NaN


In [672]:
# Corrección observación no homogénea: 
df_doct.loc[3416,'ANOIPAIS2']=np.nan
df_doct[['ANOIPAIS2','CONT2']].groupby('CONT2').agg(['unique'])

,ANOIPAIS2
,unique
CONT2,
AFRI,"[2002.0, 2003.0, 2004.0, 2007.0, 2000.0]"
ASIA,"[2007.0, 2008.0, 2009.0, 2003.0, 2000.0, 2002.0, 2004.0]"
EU27,"[2005.0, 2000.0, 2007.0, 2001.0, 2002.0, 2009.0, 2008.0, 2003.0, 2004.0, 2006.0]"
EUCA,"[2006.0, 2000.0, 2004.0, 2001.0, 2005.0, 2002.0, 2007.0, 2008.0, 2003.0, 2009.0]"
NMOV,[nan]
OCEA,"[2004.0, 2009.0, 2003.0, 2005.0, 2000.0, 2007.0, 2001.0]"
RAME,"[2006.0, 2008.0, 2003.0, 2001.0, 2007.0, 2000.0, 2004.0, 2009.0, 2002.0, 2005.0]"
REUR,"[2008.0, 2009.0, 2005.0, 2007.0, 2001.0, 2000.0, 2002.0, 2006.0, 2003.0]"


#### 'ANOFPAIS2'
Año final del tiempo de residencia en el segundo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [673]:
df_doct['ANOFPAIS2'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009'], dtype=object)

In [674]:
re_cont('ANOFPAIS2','CONT2',blank='   .')

                                                          ANOFPAIS2
                                                             unique
CONT2                                                              
AFRI                                       [2007, 2004, 2009, 2008]
ASIA                                 [2007, 2009, 2003, 2000, 2004]
EU27   [2005, 2006, 2009, 2002, 2008, 2003, 2007, 2001, 2004, 2000]
EUCA   [2008, 2002, 2000, 2005, 2007, 2001, 2003, 2006, 2004, 2009]
NMOV                                                   [   ., 2006]
OCEA                     [2005, 2009, 2000, 2006, 2008, 2002, 2004]
RAME   [2008, 2006, 2005, 2004, 2007, 2002, 2000, 2009, 2003, 2001]
REUR                     [2009, 2008, 2005, 2002, 2004, 2007, 2000]
                                                                              ANOFPAIS2
                                                                                 unique
CONT2                                                                       

In [675]:
df_doct[(df_doct['CONT2']=='NMOV')&(df_doct['ANOFPAIS2'].notna())] 
# Observaciones peculiares: periodo intermedio en España con registros de duración no suprimidos

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,Tmpo_doc,Tmpo_s/T,"(3416, ANOFPAIS2)","(733, ANOFPAIS2)"
733,90002,7,1961,07,EU27,1,EU27,None,None,2,6,0,0,0,2.0,2.0,1,5,6,NaN,1,1,2,0,None,NaN,None,0,None,0,601,1,1,0,2006,4,5,0,EU27,1985,8,1,1.0,0.0,1,40,5,0,85.0,2007.0,NaN,EU27,2.0,231.0,2.0,4.0,1.0,1.0,NaN,2.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,0.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35.0,50.0,15.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EU27,2007.0,2009.0,36.0,NMOV,NaN,2006.0,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,,,1,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,1,1,1,1,1,,1,,252,4,2,0,0,0,0,0,0,0,1,1,19.18458881,5.0,1.0,NaN,NaN
3416,410103,1,1960,01,EU27,1,EU27,None,None,2,4,0,0,1,8.0,3.0,1,5,2,NaN,1,1,1,1,01,10.0,01,2,None,0,508,1,1,1,2004,7,6,0,EU27,1983,5,1,0.0,0.0,1,90,4,0,85.0,2009.0,1.0,EU27,3.0,231.0,4.0,4.0,1.0,1.0,NaN,2.0,2.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,2.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0,265.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,60.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EU27,2006.0,2009.0,42.0,NMOV,NaN,2006.0,5.0,EU27,2000.0,2006.0,73.0,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,,,,1,,,,,,1,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,1,,,1,,1,,60,7,4,0,0,0,0,0,0,1,1,1,25.66800106,6.0,0.0,NaN,NaN


In [676]:
# Corrección observaciones no homogéneas: 
df_doct.loc[733,'ANOFPAIS2']=np.nan
df_doct.loc[3416,'ANOFPAIS2']=np.nan
df_doct[['ANOFPAIS2','CONT2']].groupby('CONT2').agg(['unique'])

,ANOFPAIS2
,unique
CONT2,
AFRI,"[2007.0, 2004.0, 2009.0, 2008.0]"
ASIA,"[2007.0, 2009.0, 2003.0, 2000.0, 2004.0]"
EU27,"[2005.0, 2006.0, 2009.0, 2002.0, 2008.0, 2003.0, 2007.0, 2001.0, 2004.0, 2000.0]"
EUCA,"[2008.0, 2002.0, 2000.0, 2005.0, 2007.0, 2001.0, 2003.0, 2006.0, 2004.0, 2009.0]"
NMOV,[nan]
OCEA,"[2005.0, 2009.0, 2000.0, 2006.0, 2008.0, 2002.0, 2004.0]"
RAME,"[2008.0, 2006.0, 2005.0, 2004.0, 2007.0, 2002.0, 2000.0, 2009.0, 2003.0, 2001.0]"
REUR,"[2009.0, 2008.0, 2005.0, 2002.0, 2004.0, 2007.0, 2000.0]"


#### 'MESES2'
Duración en meses del tiempo de residencia en el segundo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [610]:
df_doct['MESES2'].sort_values().unique() # Blank as '  .'

array(['  .', '  3', '  4', '  5', '  6', '  7', '  8', '  9', ' 10',
       ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19',
       ' 20', ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27', ' 28',
       ' 29', ' 30', ' 31', ' 32', ' 33', ' 34', ' 35', ' 36', ' 37',
       ' 38', ' 40', ' 41', ' 42', ' 43', ' 44', ' 45', ' 46', ' 47',
       ' 48', ' 49', ' 50', ' 52', ' 53', ' 54', ' 55', ' 56', ' 57',
       ' 58', ' 59', ' 60', ' 61', ' 62', ' 63', ' 67', ' 68', ' 70',
       ' 71', ' 72', ' 74', ' 75', ' 76', ' 81', ' 82', ' 83', ' 84',
       ' 86', ' 89', ' 90', ' 91', ' 92', ' 93', ' 95', ' 96', ' 97',
       '103', '104', '105', '108', '109', '110', '111', '114', '115',
       '116', '117'], dtype=object)

In [611]:
re_cont('MESES2','CONT2',blank='  .')

                                                                                                                                                                                                                                                                                                            MESES2
                                                                                                                                                                                                                                                                                                            unique
CONT2                                                                                                                                                                                                                                                                                                             
AFRI                                                                           

In [677]:
df_doct[(df_doct['CONT2']=='NMOV')&(df_doct['MESES2'].notna())] 
# Observación peculiar: periodo intermedio en España con registros de duración no suprimidos

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,Tmpo_doc,Tmpo_s/T,"(3416, ANOFPAIS2)","(733, ANOFPAIS2)"
3416,410103,1,1960,01,EU27,1,EU27,None,None,2,4,0,0,1,8.0,3.0,1,5,2,NaN,1,1,1,1,01,10.0,01,2,None,0,508,1,1,1,2004,7,6,0,EU27,1983,5,1,0.0,0.0,1,90,4,0,85.0,2009.0,1.0,EU27,3.0,231.0,4.0,4.0,1.0,1.0,NaN,2.0,2.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,2.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0,265.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,60.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EU27,2006.0,2009.0,42.0,NMOV,NaN,NaN,5.0,EU27,2000.0,2006.0,73.0,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,,,,1,,,,,,1,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,1,,,1,,1,,60,7,4,0,0,0,0,0,0,1,1,1,25.66800106,6.0,0.0,NaN,NaN


In [678]:
# Corrección observación no homogénea: 
df_doct.loc[3416,'MESES2']=np.nan
df_doct[['MESES2','CONT2']].groupby('CONT2').agg(['unique'])

,MESES2
,unique
CONT2,
AFRI,"[72.0, 48.0, 4.0, 29.0, 89.0, 104.0]"
ASIA,"[3.0, 5.0, 7.0, 13.0]"
EU27,"[7.0, 81.0, 24.0, 16.0, 4.0, 3.0, 6.0, 26.0, 36.0, 92.0, 20.0, 23.0, 9.0, 8.0, 25.0, 5.0, 12.0, 13.0, 18.0, 29.0, 114.0, 45.0, 41.0, 54.0, 62.0, 34.0, 27.0, 68.0, 82.0, 37.0, 61.0, 67.0, 33.0, 32.0, 31.0, 43.0, 10.0, 35.0, 14.0, 86.0, 22.0, 15.0, 103.0, 44.0, 52.0, 17.0, 95.0, 21.0, 105.0, 40.0,..."
EUCA,"[27.0, 36.0, 8.0, 7.0, 75.0, 9.0, 28.0, 3.0, 25.0, 5.0, 12.0, 6.0, 4.0, 24.0, 10.0, 50.0, 48.0, 71.0, 43.0, 38.0, 20.0, 11.0, 13.0, 30.0, 45.0, 31.0, 19.0, 33.0, 29.0, 26.0, 14.0, 60.0, 22.0, 41.0, 17.0, 15.0, 35.0, 16.0, 42.0, 70.0, 32.0, 56.0, 72.0, 23.0, 55.0]"
NMOV,[nan]
OCEA,"[12.0, 7.0, 75.0, 4.0, 3.0, 6.0, 10.0]"
RAME,"[22.0, 3.0, 5.0, 4.0, 26.0, 18.0, 58.0, 35.0, 60.0, 12.0, 7.0, 32.0, 72.0, 6.0, 11.0, 61.0, 30.0, 13.0]"
REUR,"[16.0, 4.0, 3.0, 21.0, 12.0, 57.0, 11.0, 5.0, 23.0, 41.0, 13.0, 26.0, 9.0, 29.0]"


#### 'CONT3'
Continente del tercer país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [612]:
df_doct['CONT3'].sort_values().unique() # Blank as '   .'

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'NMOV', 'OCEA', 'RAME', 'REUR'],
      dtype=object)

#### 'ANOIPAIS3'
Año de inicio del tiempo de residencia en el tercer país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [613]:
df_doct['ANOIPAIS3'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009'], dtype=object)

In [614]:
re_cont('ANOIPAIS3','CONT3',blank='   .')

                                                          ANOIPAIS3
                                                             unique
CONT3                                                              
AFRI                                                   [2000, 2008]
ASIA                                                   [2007, 2000]
EU27   [2000, 2005, 2008, 2002, 2007, 2006, 2001, 2003, 2004, 2009]
EUCA         [2007, 2008, 2004, 2003, 2002, 2000, 2001, 2006, 2005]
NMOV                                                         [   .]
OCEA                                                         [2004]
RAME                           [2003, 2006, 2002, 2000, 2004, 2005]
REUR                                                   [2007, 2000]
                                                                              ANOIPAIS3
                                                                                 unique
CONT3                                                                       

#### 'ANOFPAIS3'
Año final del tiempo de residencia en el tercer país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [615]:
df_doct['ANOFPAIS3'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009'], dtype=object)

In [616]:
re_cont('ANOFPAIS3','CONT3',blank='   .')

                                                          ANOFPAIS3
                                                             unique
CONT3                                                              
AFRI                                                   [2001, 2008]
ASIA                                       [2007, 2000, 2008, 2006]
EU27   [2005, 2007, 2001, 2002, 2009, 2008, 2003, 2004, 2006, 2000]
EUCA         [2007, 2009, 2005, 2004, 2002, 2001, 2008, 2006, 2000]
NMOV                                                         [   .]
OCEA                                                         [2005]
RAME                     [2006, 2008, 2002, 2000, 2003, 2005, 2007]
REUR                                             [2008, 2001, 2002]
                                                                              ANOFPAIS3
                                                                                 unique
CONT3                                                                       

#### 'MESES3'
Duración en meses del tiempo de residencia en el tercer país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [617]:
df_doct['MESES3'].sort_values().unique() # Blank as '  .'

array(['  .', '  2', '  3', '  4', '  5', '  6', '  7', '  8', '  9',
       ' 10', ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18',
       ' 19', ' 20', ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27',
       ' 28', ' 29', ' 30', ' 31', ' 32', ' 33', ' 34', ' 35', ' 36',
       ' 37', ' 38', ' 39', ' 40', ' 41', ' 42', ' 43', ' 44', ' 45',
       ' 46', ' 47', ' 48', ' 49', ' 50', ' 51', ' 52', ' 53', ' 54',
       ' 55', ' 56', ' 57', ' 58', ' 59', ' 60', ' 61', ' 62', ' 64',
       ' 65', ' 66', ' 67', ' 68', ' 69', ' 70', ' 72', ' 73', ' 74',
       ' 75', ' 76', ' 77', ' 78', ' 79', ' 80', ' 81', ' 82', ' 83',
       ' 85', ' 86', ' 87', ' 88', ' 89', ' 90', ' 91', ' 92', ' 93',
       ' 96', ' 97', ' 98', '100', '101', '102', '103', '104', '105',
       '106', '108', '109', '110', '112', '113', '114'], dtype=object)

In [618]:
re_cont('MESES3','CONT3',blank='  .')

                                                                                                                                                                                                                                                                                                            MESES3
                                                                                                                                                                                                                                                                                                            unique
CONT3                                                                                                                                                                                                                                                                                                             
AFRI                                                                           

#### 'CONT4'
Continente del cuarto país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [619]:
df_doct['CONT4'].sort_values().unique() # Blank as '   .'

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'NMOV', 'OCEA', 'RAME', 'REUR'],
      dtype=object)

#### 'ANOIPAIS4'
Año de inicio del tiempo de residencia en el cuarto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [620]:
df_doct['ANOIPAIS4'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008'], dtype=object)

In [623]:
re_cont('ANOIPAIS4','CONT4',blank='   .')

                                                    ANOIPAIS4
                                                       unique
CONT4                                                        
AFRI                                             [2007, 2001]
ASIA                                       [2000, 2002, 2004]
EU27   [2000, 2005, 2007, 2002, 2008, 2006, 2001, 2004, 2003]
EUCA   [2004, 2000, 2008, 2003, 2005, 2002, 2006, 2007, 2001]
NMOV                                                   [   .]
OCEA                                       [2000, 2002, 2006]
RAME   [2002, 2004, 2005, 2001, 2007, 2008, 2003, 2000, 2006]
REUR                                 [2003, 2004, 2002, 2005]
                                                                      ANOIPAIS4
                                                                         unique
CONT4                                                                          
AFRI                                                           [2007.0, 2001.0

#### 'ANOFPAIS4'
Año final del tiempo de residencia en el cuarto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [622]:
df_doct['ANOFPAIS4'].sort_values().unique() # Blank as '   .'

array([2000., 2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008.,
         nan])

In [621]:
re_cont('ANOFPAIS4','CONT4',blank='   .')

                                                    ANOFPAIS4
                                                       unique
CONT4                                                        
AFRI                                             [2007, 2001]
ASIA                                             [2002, 2005]
EU27   [2003, 2005, 2007, 2002, 2008, 2006, 2001, 2004, 2000]
EUCA   [2004, 2000, 2008, 2003, 2005, 2002, 2006, 2001, 2007]
NMOV                                                   [   .]
OCEA                                       [2000, 2002, 2006]
RAME   [2002, 2005, 2001, 2007, 2008, 2003, 2004, 2000, 2006]
REUR                                       [2003, 2004, 2005]
                                                                      ANOFPAIS4
                                                                         unique
CONT4                                                                          
AFRI                                                           [2007.0, 2001.0

#### 'MESES4'
Duración en meses del tiempo de residencia en el cuarto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [625]:
df_doct['MESES4'].sort_values().unique() # Blank as '  .'

array(['  .', '  3', '  4', '  5', '  6', '  7', '  8', '  9', ' 10',
       ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19',
       ' 20', ' 21', ' 23', ' 24', ' 26', ' 27', ' 28', ' 29', ' 31',
       ' 32', ' 33', ' 34', ' 35', ' 36', ' 38', ' 39', ' 44', ' 45',
       ' 48', ' 49', ' 50', ' 53', ' 57', ' 58', ' 60', ' 76', ' 77',
       ' 78', ' 84', ' 88', ' 92', ' 93', ' 99', '101'], dtype=object)

In [626]:
re_cont('MESES4','CONT4',blank='  .')

                                                                                                                                                                                                                                                 MESES4
                                                                                                                                                                                                                                                 unique
CONT4                                                                                                                                                                                                                                                  
AFRI                                                                                                                                                                                                                                         [  3,  12]
ASIA    

#### 'CONT5'
Continente del quinto país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [627]:
df_doct['CONT5'].sort_values().unique() # Blank as '   .'

array(['EU27', 'EUCA', 'NMOV', 'RAME'], dtype=object)

#### 'ANOIPAIS5'
Año de inicio del tiempo de residencia en el quinto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [628]:
df_doct['ANOIPAIS5'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008'], dtype=object)

In [629]:
re_cont('ANOIPAIS5','CONT5',blank='   .')

                                                    ANOIPAIS5
                                                       unique
CONT5                                                        
EU27   [2000, 2003, 2004, 2001, 2005, 2002, 2007, 2006, 2008]
EUCA                                             [2007, 2000]
NMOV                                                   [   .]
RAME                                       [2002, 2004, 2003]
                                                                      ANOIPAIS5
                                                                         unique
CONT5                                                                          
EU27   [2000.0, 2003.0, 2004.0, 2001.0, 2005.0, 2002.0, 2007.0, 2006.0, 2008.0]
EUCA                                                           [2007.0, 2000.0]
NMOV                                                                      [nan]
RAME                                                   [2002.0, 2004.0, 2003.0]


#### 'ANOFPAIS5'
Año final del tiempo de residencia en el quinto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [630]:
df_doct['ANOFPAIS5'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008'], dtype=object)

In [631]:
re_cont('ANOFPAIS5','CONT5',blank='   .')

                                                    ANOFPAIS5
                                                       unique
CONT5                                                        
EU27   [2005, 2007, 2002, 2004, 2003, 2006, 2001, 2000, 2008]
EUCA                                       [2008, 2000, 2007]
NMOV                                                   [   .]
RAME                                       [2002, 2004, 2006]
                                                                      ANOFPAIS5
                                                                         unique
CONT5                                                                          
EU27   [2005.0, 2007.0, 2002.0, 2004.0, 2003.0, 2006.0, 2001.0, 2000.0, 2008.0]
EUCA                                                   [2008.0, 2000.0, 2007.0]
NMOV                                                                      [nan]
RAME                                                   [2002.0, 2004.0, 2006.0]


#### 'MESES5'
Duración en meses del tiempo de residencia en el quinto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [632]:
df_doct['MESES5'].sort_values().unique() # Blank as '  .'

array(['  .', '  3', '  4', '  5', '  6', '  7', '  8', '  9', ' 10',
       ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19',
       ' 20', ' 21', ' 22', ' 24', ' 25', ' 26', ' 27', ' 28', ' 29',
       ' 30', ' 31', ' 32', ' 33', ' 34', ' 37', ' 38', ' 41', ' 42',
       ' 43', ' 44', ' 45', ' 48', ' 49', ' 50', ' 51', ' 52', ' 53',
       ' 54', ' 56', ' 60', ' 61', ' 62', ' 63', ' 64', ' 65', ' 66',
       ' 69', ' 73', ' 74', ' 80', ' 81', ' 86', ' 89', ' 90', ' 92',
       ' 93', '101'], dtype=object)

In [633]:
re_cont('MESES5','CONT5',blank='  .')

                                                                                                                                                                                                                                                                                                            MESES5
                                                                                                                                                                                                                                                                                                            unique
CONT5                                                                                                                                                                                                                                                                                                             
EU27   [ 62,  45,  22,  89,  25,   7,   9,  18,  20,  48,  31,  14,  24,  15,  

#### 'CONT6'
Continente del sexto país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [634]:
df_doct['CONT6'].sort_values().unique() # Blank as '   .'

array(['ASIA', 'EU27', 'EUCA', 'NMOV', 'OCEA', 'RAME', 'REUR'],
      dtype=object)

#### 'ANOIPAIS6'
Año de inicio del tiempo de residencia en el sexto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [635]:
df_doct['ANOIPAIS6'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007'], dtype=object)

In [636]:
re_cont('ANOIPAIS6','CONT6',blank='   .')

                                              ANOIPAIS6
                                                 unique
CONT6                                                  
ASIA                                             [2000]
EU27   [2000, 2003, 2004, 2002, 2001, 2006, 2005, 2007]
EUCA   [2003, 2000, 2007, 2001, 2004, 2002, 2005, 2006]
NMOV                                             [   .]
OCEA                                             [2005]
RAME                     [2002, 2000, 2005, 2006, 2001]
REUR                                       [2003, 2004]
                                                              ANOIPAIS6
                                                                 unique
CONT6                                                                  
ASIA                                                           [2000.0]
EU27   [2000.0, 2003.0, 2004.0, 2002.0, 2001.0, 2006.0, 2005.0, 2007.0]
EUCA   [2003.0, 2000.0, 2007.0, 2001.0, 2004.0, 2002.0, 2005.0, 2006.0]
NMOV    

#### 'ANOFPAIS6'
Año final del tiempo de residencia en el sexto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [637]:
df_doct['ANOFPAIS6'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007'], dtype=object)

In [638]:
re_cont('ANOFPAIS6','CONT6',blank='   .')

                                              ANOFPAIS6
                                                 unique
CONT6                                                  
ASIA                                             [2001]
EU27   [2000, 2003, 2004, 2001, 2002, 2006, 2007, 2005]
EUCA         [2003, 2000, 2007, 2001, 2004, 2005, 2002]
NMOV                                             [   .]
OCEA                                             [2005]
RAME                     [2002, 2000, 2001, 2005, 2006]
REUR                                       [2003, 2004]
                                                              ANOFPAIS6
                                                                 unique
CONT6                                                                  
ASIA                                                           [2001.0]
EU27   [2000.0, 2003.0, 2004.0, 2001.0, 2002.0, 2006.0, 2007.0, 2005.0]
EUCA           [2003.0, 2000.0, 2007.0, 2001.0, 2004.0, 2005.0, 2002.0]
NMOV    

#### 'MESES6'
Duración en meses del tiempo de residencia en el sexto país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [639]:
df_doct['MESES6'].sort_values().unique() # Blank as '  .'

array(['  .', '  3', '  4', '  5', '  6', '  7', '  8', '  9', ' 11',
       ' 12', ' 16', ' 18', ' 19', ' 20', ' 21', ' 30', ' 31', ' 37',
       ' 41', ' 53', ' 57', ' 58', ' 88'], dtype=object)

In [641]:
re_cont('MESES6','CONT6',blank='  .')

                                                                                                          MESES6
                                                                                                          unique
CONT6                                                                                                           
ASIA                                                                                                       [ 12]
EU27   [  3,   7,  57,   4,  53,   5,  41,  31,   9,  12,  19,   6,  58,  18,  88,  20,  21,  30,  37,  16,   8]
EUCA                                                                    [  3,   4,   5,   6,   9,  19,  11,   7]
NMOV                                                                                                       [  .]
OCEA                                                                                                       [  3]
RAME                                                                                   [  4,   3

#### 'CONT7'
Continente del séptimo país de residencia -en orden cronológico inverso- durante al menos 3 meses en el periodo 2000-2009 (Códigos: vease la variable 'CONTNACIM', y 'NMOV' si no residió fuera de España)

In [642]:
df_doct['CONT7'].sort_values().unique() # Blank as '   .'

array(['AFRI', 'EU27', 'EUCA', 'NMOV'], dtype=object)

#### 'ANOIPAIS7'
Año de inicio del tiempo de residencia en el séptimo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [643]:
df_doct['ANOIPAIS7'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006'],
      dtype=object)

In [644]:
re_cont('ANOIPAIS7','CONT7',blank='   .')

                                        ANOIPAIS7
                                           unique
CONT7                                            
AFRI                                       [2005]
EU27   [2000, 2001, 2003, 2006, 2004, 2005, 2002]
EUCA                                       [2006]
NMOV                                       [   .]
                                                      ANOIPAIS7
                                                         unique
CONT7                                                          
AFRI                                                   [2005.0]
EU27   [2000.0, 2001.0, 2003.0, 2006.0, 2004.0, 2005.0, 2002.0]
EUCA                                                   [2006.0]
NMOV                                                      [nan]


#### 'ANOFPAIS7'
Año final del tiempo de residencia en el séptimo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [645]:
df_doct['ANOFPAIS7'].sort_values().unique() # Blank as '   .'

array(['   .', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007'], dtype=object)

In [646]:
re_cont('ANOFPAIS7','CONT7',blank='   .')

                                              ANOFPAIS7
                                                 unique
CONT7                                                  
AFRI                                             [2005]
EU27   [2003, 2002, 2000, 2004, 2006, 2001, 2005, 2007]
EUCA                                             [2007]
NMOV                                             [   .]
                                                              ANOFPAIS7
                                                                 unique
CONT7                                                                  
AFRI                                                           [2005.0]
EU27   [2003.0, 2002.0, 2000.0, 2004.0, 2006.0, 2001.0, 2005.0, 2007.0]
EUCA                                                           [2007.0]
NMOV                                                              [nan]


#### 'MESES7'
Duración en meses del tiempo de residencia en el septimo país distinto de España mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [647]:
df_doct['MESES7'].sort_values().unique() # Blank as '  .'

array(['  .', '  3', '  4', '  5', '  6', '  7', '  8', '  9', ' 10',
       ' 11', ' 12', ' 17', ' 19', ' 20', ' 23', ' 24', ' 27', ' 29',
       ' 30', ' 31', ' 32', ' 36', ' 37', ' 41', ' 42', ' 44', ' 48',
       ' 51', ' 52', ' 53', ' 56', ' 60', ' 63', ' 64', ' 69', ' 86'],
      dtype=object)

In [648]:
re_cont('MESES7','CONT7',blank='  .')

                                                                                                                                                                                MESES7
                                                                                                                                                                                unique
CONT7                                                                                                                                                                                 
AFRI                                                                                                                                                                             [  4]
EU27   [ 44,  36,  23,  32,  10,   9,  27,   7,   8,  20,  37,  19,  63,  30,   6,   4,  24,  48,  17,  60,  31,  41,  11,  12,  51,   5,  53,  29,  42,   3,  86,  69,  52,  64,  56]
EUCA                                                                                 

#### 'MOTFUERA1'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿**Realizar el doctorado** fue un motivo para vivir fuera de España en el periodo 2000-2009?

In [769]:
df_doct['MOTFUERA1'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [770]:
df_doct[['CONT1','CONT2','CONT3','CONT4','CONT5','CONT6','CONT7','MOTFUERA1']].groupby('MOTFUERA1').agg(['unique'])

,CONT1,CONT2,CONT3,CONT4,CONT5,CONT6,CONT7
,unique,unique,unique,unique,unique,unique,unique
MOTFUERA1,,,,,,,
,"[NMOV, EU27, RAME, EUCA, REUR, AFRI, OCEA, ASIA]","[NMOV, EU27, EUCA, RAME, REUR, AFRI, OCEA, ASIA]","[NMOV, EU27, RAME, AFRI, REUR, EUCA, ASIA]","[NMOV, EU27, RAME, EUCA, REUR, OCEA, ASIA, AFRI]","[NMOV, EU27, EUCA, RAME]","[NMOV, EU27, EUCA, ASIA, RAME, OCEA]","[NMOV, EU27, EUCA, AFRI]"
1,"[EU27, REUR, EUCA]","[EU27, EUCA, RAME, REUR, ASIA]","[EU27, NMOV, OCEA, EUCA, RAME]","[NMOV, EU27, EUCA, RAME, REUR, ASIA]","[NMOV, EU27, EUCA, RAME]","[NMOV, EUCA, EU27, RAME, REUR]","[NMOV, EU27]"


In [771]:
def motfuera(col):
    # Se imputa respuesta No (0) si fue preguntado y no respondió Sí: 
    df_doct[col].mask(((df_doct['CONT1']!='NMOV')|(df_doct['CONT2']!='NMOV')|(df_doct['CONT3']!='NMOV')
                             |(df_doct['CONT4']!='NMOV')|(df_doct['CONT5']!='NMOV')|(df_doct['CONT6']!='NMOV')
                             |(df_doct['CONT7']!='NMOV'))
                      &(df_doct[col]!='1'),0,inplace=True)
    # Se imputa None a quien no fue preguntado: 
    df_doct[col].where((df_doct[col]==0)|(df_doct[col]=='1'),None, inplace=True)
    df_doct[col]=df_doct[col].astype('float')
    return

In [772]:
motfuera('MOTFUERA1')

In [773]:
df_doct[['CONT1','CONT2','CONT3','CONT4','CONT5','CONT6','CONT7','MOTFUERA1']].groupby('MOTFUERA1').agg(['unique'])

,CONT1,CONT2,CONT3,CONT4,CONT5,CONT6,CONT7
,unique,unique,unique,unique,unique,unique,unique
MOTFUERA1,,,,,,,
0.0,"[EU27, RAME, EUCA, REUR, AFRI, OCEA, ASIA]","[EU27, EUCA, RAME, REUR, AFRI, NMOV, OCEA, ASIA]","[EU27, RAME, NMOV, AFRI, REUR, EUCA, ASIA]","[NMOV, EU27, RAME, EUCA, REUR, OCEA, ASIA, AFRI]","[NMOV, EU27, EUCA, RAME]","[NMOV, EU27, EUCA, ASIA, RAME, OCEA]","[NMOV, EU27, EUCA, AFRI]"
1.0,"[EU27, REUR, EUCA]","[EU27, EUCA, RAME, REUR, ASIA]","[EU27, NMOV, OCEA, EUCA, RAME]","[NMOV, EU27, EUCA, RAME, REUR, ASIA]","[NMOV, EU27, EUCA, RAME]","[NMOV, EUCA, EU27, RAME, REUR]","[NMOV, EU27]"


In [774]:
df_doct['MOTFUERA1'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA2'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿**Finalizar el postdoctorado o un contrato de trabajo** fue un motivo para vivir fuera de España en el periodo 2000-2009?

In [776]:
df_doct['MOTFUERA2'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [777]:
motfuera('MOTFUERA2')
df_doct['MOTFUERA2'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA3'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿Fueron **otros factores relacionados con el empleo o económicos** (traslado laboral porel mismo organismo o empresa, cambio de empleo, garantía u oferta de trabajo, o realización de un postdoctorado) motivos para vivir fuera de España en el periodo 2000-2009?

In [778]:
df_doct['MOTFUERA3'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [779]:
motfuera('MOTFUERA3')
df_doct['MOTFUERA3'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA4'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿Fueron **factores académicos** (mayor posibilidad de realizar publicaciones, desarrollo o continuidad de la tesis doctoral, trabajar en un área específica no existente en el país de origen, creación de un equipo propio o una nueva área de investigación) motivos para vivir fuera de España en el periodo 2000-2009?

In [780]:
df_doct['MOTFUERA4'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [781]:
motfuera('MOTFUERA4')
df_doct['MOTFUERA4'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA5'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿Fueron **factores personales o familiares** motivos para vivir fuera de España en el periodo 2000-2009?

In [782]:
df_doct['MOTFUERA5'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [783]:
motfuera('MOTFUERA5')
df_doct['MOTFUERA5'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA6'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿Fueron **factores políticos o administrativos** (finalización de permiso de residencia o visado, etc.) motivos para vivir fuera de España en el periodo 2000-2009?

In [784]:
df_doct['MOTFUERA6'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [785]:
motfuera('MOTFUERA6')
df_doct['MOTFUERA6'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTFUERA7'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  
¿Fueron **otros motivos** los que le llevaron a vivir fuera de España en el periodo 2000-2009?

In [786]:
df_doct['MOTFUERA7'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [787]:
motfuera('MOTFUERA7')
df_doct['MOTFUERA7'].sort_values().unique()

array([ 0.,  1., nan])

#### 'MOTVENIR1'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fue **finalizar el doctorado** un motivo para venir o volver a España en el periodo 2000-2009?

In [842]:
df_doct['MOTVENIR1'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [843]:
df_doct[['CONT1','CONT2','CONT3','CONT4','CONT5','CONT6','CONT7','MOTVENIR1']].groupby('MOTVENIR1').agg(['unique'])

,CONT1,CONT2,CONT3,CONT4,CONT5,CONT6,CONT7
,unique,unique,unique,unique,unique,unique,unique
MOTVENIR1,,,,,,,
,"[NMOV, EU27, RAME, EUCA, REUR, AFRI, OCEA, ASIA]","[NMOV, EU27, EUCA, RAME, REUR, AFRI, OCEA, ASIA]","[NMOV, EU27, RAME, AFRI, REUR, EUCA, ASIA]","[NMOV, EU27, RAME, EUCA, REUR, OCEA, ASIA, AFRI]","[NMOV, EU27, EUCA, RAME]","[NMOV, EUCA, EU27, RAME, OCEA, REUR]","[NMOV, EU27, EUCA, AFRI]"
1,"[EU27, REUR, EUCA]","[EU27, RAME, EUCA, REUR, ASIA]","[EU27, NMOV, OCEA, AFRI, REUR]","[NMOV, EU27, RAME, EUCA, REUR, ASIA]","[NMOV, EU27, RAME]","[NMOV, EU27, EUCA, RAME, ASIA, REUR]","[NMOV, EU27]"


In [844]:
def motvenir(col):
    # Se imputa respuesta No (0) si fue preguntado y no respondió Sí: 
    df_doct[col].mask(((df_doct['CONT1']=='NMOV')&((df_doct['CONT2']!='NMOV')|(df_doct['CONT3']!='NMOV')
                      |(df_doct['CONT4']!='NMOV')|(df_doct['CONT5']!='NMOV')|(df_doct['CONT6']!='NMOV')
                      |(df_doct['CONT7']!='NMOV')))
                      |((df_doct['CONT2']=='NMOV')&((df_doct['CONT3']!='NMOV')|(df_doct['CONT4']!='NMOV')
                            |(df_doct['CONT5']!='NMOV')|(df_doct['CONT6']!='NMOV')|(df_doct['CONT7']!='NMOV')))
                      |((df_doct['CONT3']=='NMOV')&((df_doct['CONT4']!='NMOV')|(df_doct['CONT5']!='NMOV')
                            |(df_doct['CONT6']!='NMOV')|(df_doct['CONT7']!='NMOV')) )                   
                      |((df_doct['CONT4']=='NMOV')&((df_doct['CONT5']!='NMOV')|(df_doct['CONT6']!='NMOV')
                            |(df_doct['CONT7']!='NMOV')) )
                      |((df_doct['CONT5']=='NMOV')&((df_doct['CONT6']!='NMOV')|(df_doct['CONT7']!='NMOV')))
                      |((df_doct['CONT6']=='NMOV')&(df_doct['CONT7']!='NMOV'))                  
                      &(df_doct[col]!='1'),0,inplace=True)
    # Se imputa None a quien no fue preguntado: 
    df_doct[col].where((df_doct[col]==0)|(df_doct[col]=='1'),None, inplace=True)
    df_doct[col]=df_doct[col].astype('float')
    return

In [845]:
motvenir('MOTVENIR1')

In [846]:
df_doct[['CONT1','CONT2','CONT3','CONT4','CONT5','CONT6','CONT7','MOTVENIR1']].groupby('MOTVENIR1').agg(['unique'])

,CONT1,CONT2,CONT3,CONT4,CONT5,CONT6,CONT7
,unique,unique,unique,unique,unique,unique,unique
MOTVENIR1,,,,,,,
0.0,[EU27],[NMOV],[EU27],[NMOV],[NMOV],[NMOV],[NMOV]
1.0,"[EU27, REUR, EUCA]","[EU27, RAME, EUCA, REUR, ASIA]","[EU27, NMOV, OCEA, AFRI, REUR]","[NMOV, EU27, RAME, EUCA, REUR, ASIA]","[NMOV, EU27, RAME]","[NMOV, EU27, EUCA, RAME, ASIA, REUR]","[NMOV, EU27]"


In [847]:
df_doct[df_doct['MOTVENIR1']==0] #  Solo una observación (regresó por motivos distintos a finalizar el doctorado)

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,Tmpo_doc,Tmpo_s/T,"(3416, ANOFPAIS2)","(733, ANOFPAIS2)"
3416,410103,1,1960,01,EU27,1,EU27,None,None,2,4,0,0,1,8.0,3.0,1,5,2,NaN,1,1,1,1,01,10.0,01,2,None,0,508,1,1,1,2004,7,6,0,EU27,1983,5,1,0.0,0.0,1,90,4,0,85.0,2009.0,1.0,EU27,3.0,231.0,4.0,4.0,1.0,1.0,NaN,2.0,2.0,1.0,2.0,3.0,1.0,2.0,3.0,1.0,1.0,2.0,3.0,3.0,2.0,3.0,2.0,1.0,1.0,265.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,60.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EU27,2006.0,2009.0,42.0,NMOV,NaN,NaN,NaN,EU27,2000.0,2006.0,73.0,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,NMOV,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,1,,,1,,1,,60,7,4,0,0,0,0,0,0,1,1,1,25.66800106,6.0,0.0,NaN,NaN


#### 'MOTVENIR2'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **finalizar el postdoctorado o un contrato de trabajo** motivos para venir o volver a España en el periodo 2000-2009?

In [848]:
df_doct['MOTVENIR2'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [849]:
motvenir('MOTVENIR2')
df_doct['MOTVENIR2'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

#### 'MOTVENIR3'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **otros factores relacionados con el empleo o económicos** (traslado laboral por el mismo organismo o empresa, cambio de empleo, garantía u oferta de trabajo, o realización de un postdoctorado) motivos para venir o volver a España en el periodo 2000-2009?

In [850]:
df_doct['MOTVENIR3'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [851]:
motvenir('MOTVENIR3')
df_doct['MOTVENIR3'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

#### 'MOTVENIR4'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **factores académicos** (mayor posibilidad de realizar publicaciones, desarrollo o continuidad de la tesis doctoral, trabajar en un área específica no existente en el país de origen, creación de un equipo propio o una nueva área de investigación) motivos para venir o volver a España en el periodo 2000-2009?

In [852]:
df_doct['MOTVENIR4'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [853]:
motvenir('MOTVENIR4')
df_doct['MOTVENIR4'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

#### 'MOTVENIR5'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **factores personales o familiares** motivos para venir o volver a España en el periodo 2000-2009?

In [854]:
df_doct['MOTVENIR5'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [855]:
motvenir('MOTVENIR5')
df_doct['MOTVENIR5'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

#### 'MOTVENIR6'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **factores políticos o administrativos** (finalización de permiso de residencia o visado, etc.) motivos para venir o volver a España en el periodo 2000-2009?

In [857]:
df_doct['MOTVENIR6'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [858]:
motvenir('MOTVENIR6')
df_doct['MOTVENIR6'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

#### 'MOTVENIR7'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   
¿Fueron **otros motivos** los que le llevaron a venir o volver a España en el periodo 2000-2009?

In [859]:
df_doct['MOTVENIR7'].sort_values().unique() # Blank as ' '

array([' ', '1'], dtype=object)

In [860]:
motvenir('MOTVENIR7')
df_doct['MOTVENIR7'].sort_values().unique() # Blank as ' '

array([ 0.,  1., nan])

In [ ]:
df_doct['PREVIRSE'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MESANTESIR'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE1'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE2'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE3'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE4'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE5'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTIRSE6'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['CONTOUT'].sort_values().unique() 

In [ ]:
df_doct['INV'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV1'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV2'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV3'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV4'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV5'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV6'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV7'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV8'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV9'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTNOINV9'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV1'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV2'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV3'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV4'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV5'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV6'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV7'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV8'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MOTINV9'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['MESESINV'].sort_values().unique() # Blank as '  .'

In [ ]:
df_doct['ARTIC'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['LIBROS'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['NUMPAT'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['PATNAC'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['PATEU'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['PATCOOP'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['PATCOM'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['COMPA'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['TUTOR'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['COOPERA'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['INVFUT'].sort_values().unique() # Blank as ' '

In [ ]:
df_doct['FACTOR'].sort_values().unique()

In [688]:
for col in df_doct.columns: 
    print (col, df_doct[col].dtype)

MUIDENT int64
CCAARESI int64
ANONAC int64
CCAANAC object
CONTNACIM object
RELA int64
CONTNAC1 object
CONTNAC2 object
CONTNAC3 object
SEXO int64
ESTADOCIVIL int64
DEPEN5 int64
DEPEN18 int64
DEPENMAS int64
NIVESTPA float64
NIVESTMA float64
NIVPROFPA int64
NIVPROFMA int64
ACTIPRIPA1 int64
ACTIPRIMA1 float64
EDUPRIM int64
EDUSECUN int64
BACHILLER int64
RES int64
CARES1 object
MESRES1 float64
CARES2 object
MESRES2 int64
CARES3 object
MESRES3 int64
CODIDOC int64
INFUN int64
INAPL int64
DESTEC int64
ANODOC int64
MESDOC int64
DURDOCANO int64
DURDOCMES int64
CONTEST object
ANOEST int64
FINADOC int64
TRABDOC int64
ANOSINTRA float64
MESSINTRA float64
SITLAB int64
HTRAB int64
INGRESOS int64
AUTON int64
CNAE2 float64
ANOINIALP float64
CAEMP float64
CONTEMP object
SECTOR float64
CODALP float64
CATEGPROF float64
HLECT float64
TIPOCONT float64
JORLAB float64
BUSCATRAB float64
NIVELMIN float64
NIVELEST float64
RELTRABDOC float64
NIVELSAT1 float64
NIVELSAT2 float64
NIVELSAT3 float64
NIVELSAT4 float64
NI